## Tensor Decomposition 

In [1]:
#All the imports for the program go here
import time
import sys
from os import chdir
from os import getcwd
from os import path
import os
import math
from collections import Counter
import numpy as np 
import tensorflow as tf
from ktensor import KruskalTensor
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from sklearn.metrics.cluster import v_measure_score
from scipy import spatial

## Please change directory which has the set of malware and benign files!

In [2]:
# This block takes the present working directory 
oldwd = getcwd()
#dataDir = oldwd+"/LZsetUpdated/zeusBinary/"
#dataDir = oldwd+"/TermDocumentTensor/test_files/"
dataDir = "/Users/Phaniteja/desktop/RA/latestTestFiles/"
print(oldwd)
text_files = [f for f in os.listdir(dataDir)]# if f.endswith('.exe')]
numberOfFiles = len(text_files)
print("Found %d files to look at" % numberOfFiles)

/Users/Phaniteja/Desktop/Github/Tensor-Decomposition/NotebooksWithExe
Found 9754 files to look at


In [3]:
unpackedFiles = {'ctrl2cap.exe', 'efsdump.exe', 'zeusbin_0b02e6141fdeb045a5475fc423d2dafe.ex0', '.pkl', 'Coreinfo.exe', 'devmprov.exe', 'zeusbin_0cfab1f28f6751dee14ff014bdbc3ac2.ex0', 'disk2vhd.exe', 'bt.exe', 'AccessEnum.exe', 'zeusbin_0b748a4f8636a80ea12015507b6ae1bc.ex0', 'accesschk.exe', 'zeusbin_0bdea24bead430c14df16cd6d8d3432b.ex0', '.DS_Store', 'csharpexec-test.exe', 'ADExplorer.exe', 'zeusbin_0a64f9e0e3751f1c931149c54a19c707.ex0', 'ADInsight.exe'}

In [4]:
unpackedFiles = {'zeusbin_0097e7ebc48af545d0758c9f2fb6882e.ex0', 'zeusbin_62169455fd9b0908eb51191e38217c4c.ex0', 'zeusbin_3302ed4cef0e8af6e7dae40b011ea772.ex0', 'zeusbin_5a8108521f05eeea5f4b0e3043edf782.ex0', 'zeusbin_f8292ab24afb9afb1d2c5f2a37dbc0ae.ex0', 'zeusbin_57e06fdb4bb8db21c2e6b8a0017a8378.ex0', 'zeusbin_6d008d3d7c6925bec035d4dad8d47b66.ex0', 'zeusbin_8ef75794ed565e2512fd0a54df04e990.ex0', 'zeusbin_34a40ad0c257045d11ddde855b45fa66.ex0', 'zeusbin_53f2772061376525ce614fdad8c72111.ex0', 'zeusbin_403f661916fb74386582cdb36ebd9320.ex0', 'zeusbin_cacc70f6b95519e2d9a1c76dce3f7ca3.ex0', 'zeusbin_9c8c705051f0bf9d6ba9be1745090ef6.ex0', 'zeusbin_b9d5e13b7aee8645c73b63c1d73e7550.ex0', 'zeusbin_8e472d33b1616a3e9a036d99e2177b08.ex0', 'zeusbin_e265b2c11f928c5465cffa5188a8c3ca.ex0', 'zeusbin_362895a0d94394f9b98e0d61235cdb6b.ex0', 'zeusbin_09b22d4e11817e2652b205686c57841c.ex0', 'zeusbin_b863338d9d2b603faa4469f3b1ff5e6a.ex0', 'zeusbin_a204bb60062b4570214d3565634fa257.ex0', 'zeusbin_6680786cb8fe6eec79232ef69c876b6e.ex0', 'zeusbin_909e644c2e21b73b476e9ae888111584.ex0', 'zeusbin_ce0e4be45919462647e19f526a26b6e9.ex0', 'zeusbin_20dda594b415f1c1b3b1215de5d6db05.ex0', 'zeusbin_3d76812520c74833d3a16c2a4849160e.ex0', 'zeusbin_74de8cebc0b66c7516e06a35398fe0e0.ex0', 'zeusbin_c6ca82f4af53c4ba08cfeea57f27fc16.ex0', 'zeusbin_c9cc24c3633661cd1739513f0cc0aa84.ex0', 'zeusbin_04a52158af1ce913ed5c7e174f406049.ex0', 'zeusbin_a7a466e0aac0b7b79f0c15f77dec4df0.ex0', 'zeusbin_887cd71f5c491f50d8b317e4b6f42c1a.ex0', 'zeusbin_90b612502670ee8a453ba190ec6e75a5.ex0', 'zeusbin_aa9a41cd6e8d8e2a59ad6b06666fd2d0.ex0', 'zeusbin_20f60acdf4b4bf46e1351b0530991dcd.ex0', 'zeusbin_8fab18d966599fc977cb75f1216ad327.ex0', 'zeusbin_8f16094f40acc7425f5f036cf97d05d8.ex0', 'zeusbin_52ab4a86cf8402d4ea6e86b45360d081.ex0', 'zeusbin_0ee321b3e4a47dba44f7bb14fb6b9efe.ex0', 'zeusbin_ce0dc1ab104d69ef12337cb60e100501.ex0', 'zeusbin_5ce5b500d22146ccaf1976bb2a52e6da.ex0', 'zeusbin_8e04cfba1315c3e122a4eeaa8489f4a0.ex0', 'zeusbin_fc990e6dfa5c53850c6d2f36b0c2ab48.ex0', 'zeusbin_a25263c96b548b76031d96b43fe46b08.ex0', 'zeusbin_756447e177fc3cc39912797b7ecb2f92.ex0', 'zeusbin_cbd463bf72564da7f7631f133b7d7a16.ex0', 'zeusbin_3d132253420ef5efd2f3bc99462bfa2e.ex0', 'zeusbin_506ae5000c07ca11d643da15cb8173ba.ex0', 'zeusbin_a36ac29bdce56a9822fb94c76f5d6303.ex0', 'zeusbin_1b4c833e1a18d0c5240d6694cbd5f25f.ex0', 'zeusbin_c154d321487d8d9502bcaa739bba49ae.ex0', 'zeusbin_14d294fbfef36c063b96fcfb0d849d46.ex0', 'zeusbin_12707d8923ab9d86079d89480de97c3f.ex0', 'zeusbin_c617fe4b0c9d11af6cb8e1bcaa61cd54.ex0', 'zeusbin_a03f9e8a3e8067139ad6e0170a573915.ex0', 'zeusbin_0f50312e20f6b5f2ff1ec5bbdfca4ee3.ex0', 'zeusbin_252d59c765189e7a5ad1309f49724188.ex0', 'zeusbin_9657febc71ccada0e9cfae5871f589f3.ex0', 'zeusbin_89f79e297e439304cc26519eab772bb3.ex0', 'zeusbin_c265cbc422defd00cb2c0e5671cb407c.ex0', 'zeusbin_d992a9460b72067dbccb75a1cce02639.ex0', 'zeusbin_5c8875eba48586d8f1a3b3a6e86de426.ex0', 'zeusbin_9b749134695a6e28e29cf96fca17f11b.ex0', 'zeusbin_d57672d26036ebd1586a05734d5be6bc.ex0', 'zeusbin_4d7f45666f2af51474beefc0838b351f.ex0', 'zeusbin_c8eb57fad3a9663f72fc3a6038ff0e7f.ex0', 'zeusbin_144b49b2b46195e9864aa9f317199bfb.ex0', 'zeusbin_4e1aa174471703ff67f7344ad744cffb.ex0', 'zeusbin_c9e0fb04d933a98c6f1ca6a4cd52a0f8.ex0', 'zeusbin_33098ef529c83d81e9b82ab7a0a03252.ex0', 'zeusbin_2c9067ac187db40dbf1948f50db1f185.ex0', 'zeusbin_12c1d32952f88b64990babf52936d023.ex0', 'zeusbin_57af12ad1138fd6fc45809adca5c7748.ex0', 'zeusbin_487b86fe3cd9cd0e48da7134c0857567.ex0', 'zeusbin_0ffa350c0bc96ef1368b3d2279f3378f.ex0', 'zeusbin_d333bc2fbb7c425f673f00ed928973a0.ex0', 'zeusbin_70e7e6413fa1957cb9dc85e8910eed65.ex0', 'zeusbin_0cfab1f28f6751dee14ff014bdbc3ac2.ex0', 'zeusbin_e4c0248128fe8a362cf06c680c05365d.ex0', 'zeusbin_114446798966209506ed21c9ff926d72.ex0', 'zeusbin_e7df872dbc5381f3596ea5470b4fdf35.ex0', 'zeusbin_4fb0dfc902c9bbea68d06c1d42866b53.ex0', 'zeusbin_122e910f20c37f121060d24fcb16d783.ex0', 'zeusbin_1da73f1c22739a13311f6e9807b3341b.ex0', 'zeusbin_2b8e48c443c15c33986b754b76bdc8fb.ex0', 'zeusbin_e1cd3a4bf381a26ad785357ba4b099de.ex0', 'zeusbin_d95f29db5d8933fd8894dc13b8bed726.ex0', 'zeusbin_59fc670b36704fdcf6bfec41f21b7853.ex0', 'zeusbin_c6ae73fb4cd26ef9473b0bc7fca681a9.ex0', 'zeusbin_520f2694b5b775af0da1ce71bc4fb39b.ex0', 'zeusbin_61b05811d680bc6490ca47bd2c90823a.ex0', 'zeusbin_1486e2c21a58f44ff14c9cef8da386fb.ex0', 'zeusbin_d2e40f6239c358d083d147dd1d9196dc.ex0', 'zeusbin_9a3d52668c155c36b059c92054c789d5.ex0', 'zeusbin_df7818ef37eafcbf7f95e3f262f921b7.ex0', 'zeusbin_6709f99c91e762884d3e45c8c0d0fa98.ex0', 'zeusbin_6ed709ceac4f15ea0b0542d3aedcf3cb.ex0', 'zeusbin_6395582467cce9bef97fc9a93bff739e.ex0', 'zeusbin_6333c5c2c7091c5c26aca372b041755d.ex0', 'zeusbin_193d7be9ff2a18b10dcd75072334a54f.ex0', 'zeusbin_da6fe0ff31cdfd4a6e861bfb7c0c5c91.ex0', 'zeusbin_b220a7bc97e21d1a13a08ead5cf90116.ex0', 'zeusbin_ec9fafb71cfec5ba384c4d342f2a76f4.ex0', 'zeusbin_b0fc8ef8bee2c79eb37f6880b06381d5.ex0', 'zeusbin_912e4c3a98b7b77fa8cadb8ed77b9846.ex0', 'zeusbin_15495577934562a08d8ab9a11be3ba62.ex0', 'zeusbin_3a02cb0a4bd6936206590da902db5ffb.ex0', 'zeusbin_c174ca2d54ee1d2f2439a44ac6b19e09.ex0', 'zeusbin_251c4c1501ea7d073c03cfb9077ee290.ex0', 'zeusbin_803f6e85d809e601360c22cb80a3e8f2.ex0', 'zeusbin_b0145344ea4ad267efbc653b2992974c.ex0', 'zeusbin_a830e6782611e8e29b468918d83c54c3.ex0', 'zeusbin_ca21d6561951fc87cbb0c3a55c9159e6.ex0', 'zeusbin_79189537e13971cabc64a47a98454909.ex0', 'zeusbin_729b9e4d7de555236f66455bf1186833.ex0', 'zeusbin_77bf953b707cd00f5cb5130c6b308e59.ex0', 'zeusbin_9e1b39a820e86f426475bb6651293164.ex0', 'zeusbin_9750ab9b939fada0691388a13e55c156.ex0', 'zeusbin_6a855c130d08a08295246d73a3d79af9.ex0', 'zeusbin_1b00452dd589cf71591a5f9943c0ab93.ex0', 'zeusbin_904553e47f7ab4f6e1bfa7502327303e.ex0', 'zeusbin_6b187ac48c93d272bd192a4b8e416905.ex0', 'zeusbin_e808a684802c67aa513a8e1a4056643a.ex0', 'zeusbin_8f6917df893caa43cd84b9c42df3f8b4.ex0', 'zeusbin_1665a8fc9ed420a9870588348492cad4.ex0', 'zeusbin_e4c3ab8bc9a382e281cba79961a4d6d5.ex0', 'zeusbin_d63d8f3339feb3e3077223ce0029f74d.ex0', 'zeusbin_1fbfc4e242da54b1b605a22df9ea36fd.ex0', 'zeusbin_bb37ff859cce2d7c5faa6b6b1301835b.ex0', 'zeusbin_72461bfee0808356fe9f0e8573af7650.ex0', 'zeusbin_12b46389be1acce5142619192e3d33da.ex0', 'zeusbin_cf628a9f53c82036fe644a39de7292ee.ex0', 'zeusbin_517cbe2453b6d2e4cb0418eaffa94a53.ex0', 'zeusbin_93ac7005ad34a78f4f081b8a72885e54.ex0', 'zeusbin_e5ca06d9055a3d5f2af769c54ec4136b.ex0', 'zeusbin_a460c6f9ae5540fca87033a6de47cda5.ex0', 'zeusbin_814ae4263ecb3ec7f6c70f4dcf5747e9.ex0', 'zeusbin_bfd636829520bc53852f7bcaf0530ff5.ex0', 'zeusbin_51dd42665519e92bd4fda4790afda309.ex0', 'zeusbin_ebc76f2f21d462941f33d1962769fc9c.ex0', 'zeusbin_83560514a6a3183d836fabf4c69b0732.ex0', 'zeusbin_4521bef0e4020f8c79253ee97af97cd7.ex0', 'zeusbin_a2c0171a4739242b695976e73376c0da.ex0', 'zeusbin_2420fb71d32b61b886177b20b4159a7f.ex0', 'zeusbin_5206c204a880215fcaec7104017807e1.ex0', 'zeusbin_e4287db3afa84062d5c804af7108c652.ex0', 'zeusbin_9d3f74140b5f7e957b2b9c6ec4a0cba4.ex0', 'zeusbin_51bb2b2a8a4315793e5c8075946ee30b.ex0', 'zeusbin_38029aaa03ca6f20bb0d67f789aa4923.ex0', 'zeusbin_c7e70eb5f665eb5a32101deed7d14e4b.ex0', 'zeusbin_267eb35ffc49d8721d0c29e8a5542c8f.ex0', 'zeusbin_04c235c1dd6a91f86d72b785e85d2692.ex0', 'zeusbin_ef4564964d6b66bd2f8db5c633475b67.ex0', 'zeusbin_981bc042cd2ba4f34fc0de3d927a4a73.ex0', 'zeusbin_4e6d7f70f9e23a774910e6649c127078.ex0', 'zeusbin_d5cede2be86794a4618b50e74821c01d.ex0', 'zeusbin_1db382dae4f2b3a84117245695697f97.ex0', 'zeusbin_8ddef4e68e8495c0dc77a5b55655dbdd.ex0', 'zeusbin_bec7595c4b5377921474293bb6075311.ex0', 'zeusbin_83cfcd0e8bd653c8aaf2bdfcd936edb1.ex0', 'zeusbin_04aa0d2c110d62d24cb92622755cfacd.ex0', 'zeusbin_a92d6e402e300976c4bd9c7c37742e21.ex0', 'zeusbin_fce31ac6f04df886b5c06f3c50ad7729.ex0', 'zeusbin_eea8c0976aa9d464d0bd0112247acf96.ex0', 'zeusbin_e834f5a83db82b75205eb8617aff2db9.ex0', 'zeusbin_2de8e8f9761dfb0b01b871155de46bbb.ex0', 'zeusbin_de95c43229e5b0ba4a284683ccce2d78.ex0', 'zeusbin_6dd76848f68b9acd2d805163c85f8d5f.ex0', 'zeusbin_68b329da9893e34099c7d8ad5cb9c940.ex0', 'zeusbin_ea041dae5e21d491f01d7113674de6df.ex0', 'zeusbin_a93dfbff5735c7afee4857c2a432a488.ex0', 'zeusbin_1febf89c92b7b4ddd697d4c22ae0d16a.ex0', 'zeusbin_a4ae956fb4e42c5f2eb3029cf38476f7.ex0', 'zeusbin_1c5ac7502a3bb3e74459fc50af0e5221.ex0', 'zeusbin_ac1924b156a0a7c213eb174a9c2249e7.ex0', 'zeusbin_62d609084169dbb95f4ca333435e05d0.ex0', 'zeusbin_f6016a7e2df1774fcd35459302b9f060.ex0', 'zeusbin_c1c2ac433b106b57080789ce4bcb89b9.ex0', 'zeusbin_4c48875c464b2b5944d7959bc8a5cb0d.ex0', 'zeusbin_74da39ea8ba4fd490bf2f666a3a3ab05.ex0', 'zeusbin_e0b8a8a12af18f15e69cac5edc631b50.ex0', 'zeusbin_10bb6b5fc6678e51ab9aa090e3b23595.ex0', 'zeusbin_b2193bcd49f155e5aca54a0fdb36d75b.ex0', 'zeusbin_78efdd02b00ad78fac2e14f0014cc4bc.ex0', 'zeusbin_149a14112e95576ebc24ffb8e28261df.ex0', 'zeusbin_87bb648f5c59fc75a0b55d4e1a72a262.ex0', 'zeusbin_8e065372f9a85d580742e296a0110114.ex0', 'zeusbin_174f31e04800c1384a1a6c2f88c42eee.ex0', 'zeusbin_a46ff85867847cdbefa8c343975e71ca.ex0', 'zeusbin_12b9f8583ec478d7bf7c4c3b10839b43.ex0', 'zeusbin_da27b8578c4e8ada1cfa17fb25fa161f.ex0', 'zeusbin_4b11ffde267d271b264da95b3ca89a7e.ex0', 'zeusbin_1e1450ea831c148d34c8d3d75bc2eaf0.ex0', 'zeusbin_715a907da639533bbb27ecbced1b03fb.ex0', 'zeusbin_b1c06fc49699bb178ec078add0d336cc.ex0', 'zeusbin_086089fc98f3afbc9ff2cda8dda80942.ex0', 'zeusbin_e46be0d5e5baa48ade7378a27fc3041f.ex0', 'zeusbin_eef6d9b841edcb0672660b53a49a4316.ex0', 'zeusbin_e172d09613dc8e32b1448ae800a10f86.ex0', 'zeusbin_9490cf8184989d89502c9c7624572f75.ex0', 'zeusbin_b2b1325cbfcc52be62e38fb99713b1f3.ex0', 'zeusbin_079019625e44d3802d94f83cc73b83f6.ex0', 'zeusbin_83eefb4ae7dfde2b265549e2c60ce4fa.ex0', 'zeusbin_f5389a8ee6d0860d9eaf8ddcb044be73.ex0', 'zeusbin_49eba7529172f6dbdf8a87b712cdb645.ex0', 'zeusbin_8730b28e8cc20091509b617d856c94f9.ex0', 'zeusbin_2ffac0ebcbbe9f32df017d7f099020f8.ex0', 'zeusbin_2279de832112c4aec0972cf688c64216.ex0', 'zeusbin_af2c35c0ca5ee1097f3a782b1bb1cfa4.ex0','zeusbin_91add03073c644bfd35732fc43810189.ex0','zeusbin_68f7f2dd30dc480579557780333a8bf3.ex0', 'zeusbin_97675eb3f268048604dc5155511a2a4d.ex0', 'zeusbin_70cedaff5a8ea3a95c73a3986e0e5c87.ex0', 'zeusbin_5fc108a6f2b8424e8e310b88c886bfe9.ex0', 'zeusbin_2f3b9a2525a91f3c06a54332b0074e21.ex0', 'zeusbin_1480cca97da7ae5c2b63d176d2442ceb.ex0', 'zeusbin_c22cb47aa0ec07a506c8e54478e4d83e.ex0', 'zeusbin_4ecf15b28c9a99860f8129cec979a4a5.ex0', 'zeusbin_660181d71cf48cf4fc6ba435a4ec0dd7.ex0', 'zeusbin_101eac98f8226e3e3ee8901be06eb6d9.ex0', 'zeusbin_16ab81b2c997bff0154a9b7f92f6503d.ex0', 'zeusbin_94d85dd9e0f5f16ca285552afe30a174.ex0', 'zeusbin_677a4256e0a7e45ab337d5dc9fb89c2a.ex0', 'zeusbin_8f331ea71f67a4a4e3e8e1463be0c92d.ex0', 'zeusbin_77be5a8ea34c47eb803459d91a3730fc.ex0', 'zeusbin_9098c057091100cfb3d80ee99a7add4b.ex0', 'zeusbin_cd43fa8c663e6f0049c35c23a1d06a6e.ex0', 'zeusbin_61030ca7196ae04b50de66015f6f7c38.ex0', 'zeusbin_a486edd5d966fd167f9d8fa94087913e.ex0', 'zeusbin_05a9cd49dd0b3b6cc1a7e48464999b9e.ex0', 'zeusbin_3a5864ecb258afb33ecc964a093b0336.ex0', 'zeusbin_e164aabe89fcd8d92040a15f76c995c9.ex0', 'zeusbin_3b7b7b62445f5ce8f930cb474ecbfe37.ex0', 'zeusbin_2812ac76b416428c0ed7adf7490d0133.ex0', 'zeusbin_f525a5decafcdf99ae44cd76fe9158b6.ex0', 'zeusbin_77b2c7e0c6a0b5ce2ccf46cbdb8ec1a3.ex0', 'zeusbin_a3ffae6ec54c2d923c3901cb3b90269d.ex0', 'zeusbin_9c8e0593ba342152a39c1a6d38740218.ex0', 'zeusbin_d9dbb28c6af64869b79359e020642efe.ex0', 'zeusbin_d15305d7a4e34e02489c74a5ef542f36.ex0', 'zeusbin_7adb5762997379b2c007f7e93839dfe3.ex0', 'zeusbin_43cd2fd09b821afeb1a96933f9f1d6a8.ex0', 'zeusbin_2e17b7b37870469d65b938ceb76d457c.ex0', 'zeusbin_69ef65e895d8768d145fa20ce37c3701.ex0', 'zeusbin_cf585f2785990937d3d0b1750216df11.ex0', 'zeusbin_2a662f283c6d9fceb1c473aaf9175bd0.ex0', 'zeusbin_49790b8b6c07e330aa343555cbba048a.ex0', 'zeusbin_f7f7e615ee8c7c3aff2cb69e8e4b1bee.ex0', 'zeusbin_f3ea5c3e8991ebe87bd2549d64e09765.ex0', 'zeusbin_10531148760a98a5441ce93f4432561c.ex0', 'zeusbin_79e704e49bcbc5d9f30fba9fdaf3e4b8.ex0', 'zeusbin_d9d8d6a282e30fdad5ef55266c0d4111.ex0', 'zeusbin_bf7041da35376da532ac8be3279aeaf5.ex0', 'zeusbin_94c7358e46c9ebc7c90af9e7dc3dd8ed.ex0', 'zeusbin_74f2c955e576fa1856702f5a5a70dffe.ex0', 'zeusbin_28e7837eaa3d39e6f76ee16136d02aa2.ex0', 'zeusbin_79462f574b7b022869050089f7d3f0d2.ex0', 'zeusbin_d996109a926ef4092b37288dd030d2e3.ex0', 'zeusbin_fc921d17d55525e066a3ed9d9d664f3f.ex0', 'zeusbin_ee5d6f9e4625be005dab2b599c3ba968.ex0', 'zeusbin_68b80a17f61a19570fbbb587e715303c.ex0', 'zeusbin_b2f144fc2891acad31d92593e564ae0b.ex0', 'zeusbin_b67dc1cbcb23ccef1a0328f152bf53a5.ex0', 'zeusbin_8c1c88dda615ec741ae8880acc5e3934.ex0', 'zeusbin_e5f5bc6c6576e43b11cb0ade1823d3d3.ex0', 'zeusbin_fa519bfd49ba82acd616cb12a82f998c.ex0', 'zeusbin_750e5d1723422782e1d41e31f2287965.ex0', 'zeusbin_c4b8c4cbbd8c7215b688058cecea8a2e.ex0', 'zeusbin_c89343eb3555eb425cd8d1c6c837889e.ex0', 'zeusbin_8c9952a9adb62b36381d29aa51901eb9.ex0', 'zeusbin_5b4f372d1a638d780ab160448eecf65d.ex0', 'zeusbin_b1abbf464d7c4a4ac15acd234ee5484f.ex0', 'zeusbin_697fce300c9878d15e4306093b75be3d.ex0', 'zeusbin_8ef7ab8b343a4cacd4027a4b990c623d.ex0', 'zeusbin_0504941110dd0449ce2de9e829b6316e.ex0', 'zeusbin_16097db59de7255053b0ee5ff7a7b763.ex0', 'zeusbin_dc5eb9a0b0adc1afbc0d10a4a6d0c567.ex0', 'zeusbin_8029dc7e003e25cd7b7488ddc97fbb24.ex0', 'zeusbin_a0faffef6ffdb13cfe95408e9ca8a0ea.ex0', 'zeusbin_924830d720076f9366bcd6a89a392845.ex0', 'zeusbin_385239507f00bbaffdc1136fd7bbe73b.ex0', 'zeusbin_27d2ed6678e1a6f736e1c9a1dd76e0ba.ex0', 'zeusbin_bebe31d157e5fa35a9a2be3e28852f5c.ex0', 'zeusbin_8a92fa240436cfcca099d1957e3f6e6a.ex0', 'zeusbin_a976ac3bb8db42288ba101bc53e8ead6.ex0', 'zeusbin_62316b2f638dd4c43b9da30935ba5261.ex0', 'zeusbin_647a2c43e508323da8f3a013e0de1c7a.ex0', 'zeusbin_bb62c9fabe4ac9bca4d851d1261c279c.ex0', 'zeusbin_ab6588df7bf80307986cabc764080478.ex0', 'zeusbin_33ea76477b0e08215f03c6e383bf8359.ex0', 'zeusbin_59c440c05b16118b4e71be067f0f0849.ex0', 'zeusbin_f90e52d0e6aa39ad91c94ffef5502975.ex0', 'zeusbin_67afbc78df726b6ba198f856843f84a9.ex0', 'zeusbin_318e5df01d5ebe86798731f1c33a6578.ex0', 'zeusbin_0ed58ea5eb0d4e63ff06ca0ee07dea23.ex0', 'zeusbin_fd813555c5b26fc719e782420828fd67.ex0', 'zeusbin_c835af3a25b44f4f6243f9dfaa330844.ex0', 'zeusbin_4de9efe2683504d5b5ee297372526151.ex0', 'zeusbin_03fb56bf933d7f0a81d37a599517602a.ex0', 'zeusbin_3028e0638275ba8979f551812e0793c6.ex0', 'zeusbin_10c6568ab0e5b7bf4e4aa591dae8bfdb.ex0', 'zeusbin_08d01c593a3f1c35a7595fb2294a1b8c.ex0', 'zeusbin_6172e6278c2f0a012a6aca5a5069b221.ex0', 'zeusbin_8d096ee74a1df2862c89a5845673ed58.ex0', 'zeusbin_1dfe9cb8538a519092870e2c7bace705.ex0', 'zeusbin_a8a3e535cc3b54363b7d5764a71302b6.ex0', 'zeusbin_148f868f3718ac83a9b77bb53ecf7cbe.ex0', 'zeusbin_9e0f3c1c29a60a21d85b881ab9858dfe.ex0', 'zeusbin_3e65032595fae332524a94fc0aa52edb.ex0', 'zeusbin_c69999d0c0bc520e366f551727d5937d.ex0', 'zeusbin_dd836822645f8f2a70f6fe7b54235187.ex0', 'zeusbin_3caaab49b856d8f02b7959e08eb33086.ex0', 'zeusbin_5f6a44d8559707942f29f9fc0c7efcdc.ex0', 'zeusbin_c1d417f2143b222de395b813eb951bb6.ex0', 'zeusbin_f67d176eaa0d2df796fd3daa3e6ccabc.ex0', 'zeusbin_4526c2dcf11bcc1b2db868d96b8f4463.ex0', 'zeusbin_440af60cdcd6addc29ba04616715738b.ex0', 'zeusbin_8619f6afa4f4e3ac9c712fbde92eefcb.ex0', 'zeusbin_f986515cd2d1a0d2611ec84b3f4006fd.ex0', 'zeusbin_0b748a4f8636a80ea12015507b6ae1bc.ex0', 'zeusbin_f0ca232e22a5b045741417df32176892.ex0', 'zeusbin_71f6590bcaf16d9203e9428e545bc5cb.ex0', 'zeusbin_c9f958989c1f1db0979d386795effa0d.ex0', 'zeusbin_68bed8dbfefd986c2c8b66d9eb290181.ex0', 'zeusbin_3ef0eea1cc53703cabd283fcc4b0e57a.ex0', 'zeusbin_bed20d7bf94d4a837e675557bf6bbd3e.ex0', 'zeusbin_cdd911de2b50e36566b483f3b3c1e25c.ex0', 'zeusbin_129e271c36cbd083c1251aa08f48dc25.ex0', 'zeusbin_68676213eaf9c93cc99564c7d6951cd2.ex0', 'zeusbin_1aeb1def75f13a5b590586606575cbd1.ex0', 'zeusbin_b24a4a367060d83cea54f1a26b7f9532.ex0', 'zeusbin_64f5a6c9c725cd7b1986164bd3afe0d1.ex0', 'zeusbin_dec1a97cd5d24430e2d9b2cfb8f6864d.ex0', 'zeusbin_23004c619a695b5d78fb435dd6f2442c.ex0', 'zeusbin_c8624da6815dc4451d0b717bd9e6e34a.ex0', 'zeusbin_c1669f678a5942d314611e79e7deeade.ex0', 'zeusbin_a182b445ff83d46d593b39f99a791a9a.ex0', 'zeusbin_d835bc682e092f4c968b9512f99081b5.ex0', 'zeusbin_1e233a7613b8c8ae9e077b6a217f9f62.ex0', 'zeusbin_b91fbf7db78c52aefc6506a425755440.ex0', 'zeusbin_5a06a69cb78215c99f469f8977f15d82.ex0', 'zeusbin_d490a173c8ba38d0390126d02ce773c8.ex0', 'zeusbin_bf27ac5c832d929e94066a1320ba3e95.ex0', 'zeusbin_dd84d7393e8c2f7d140eaf07e518fff2.ex0', 'zeusbin_ad3d733620917bf92e121db01a529fcf.ex0', 'zeusbin_0b02e6141fdeb045a5475fc423d2dafe.ex0', 'zeusbin_565d51366441e800b424bb3927f99f97.ex0', 'zeusbin_8e316d144823751eb0af131845684a62.ex0', 'zeusbin_369e4e031925b1e341482e630be93731.ex0', 'zeusbin_3075c65a964d58c2c17596c5af6713bf.ex0', 'zeusbin_408f3ceb086db1f75b758c9a8800e848.ex0', 'zeusbin_5002d30daaebde5748abf1681cd9694b.ex0', 'zeusbin_695b42f564eec7c3527e322da77e2947.ex0', 'zeusbin_ad3cfca22c1bc9610651ec2bc93a4e69.ex0', 'zeusbin_2d8a1f78f8a6e9f11042322ed64143af.ex0', 'zeusbin_b375c578a7cc01de1f8fe103ef74a0e1.ex0', 'zeusbin_aff3977dbd5a3a8b075d61c99c10c076.ex0', 'zeusbin_942ad1e140ea248918b9d7151dd0fdbd.ex0', 'zeusbin_d270cb1b95d1fee59e32ae19121f82d6.ex0', 'zeusbin_8ea3c9fd270c1dd58805b444a599b069.ex0', 'zeusbin_d978af223bf2e440261db4d5de52ddea.ex0', 'zeusbin_885ebb6a97edc31d95ff361a68a7b96e.ex0', 'zeusbin_d3b35fa888ed3bd1e5783c1af652bb66.ex0', 'zeusbin_cdd1c50b94586ca8aa26591b5831c4b2.ex0', 'zeusbin_e4f0fd1fe6a1903245a2d709f6e55bfb.ex0', 'zeusbin_1377860c072645b87de4c2b95fda5a31.ex0', 'zeusbin_cb69538063aab7be10edeb4a122fd97a.ex0', 'zeusbin_a599ce1912510f021bf490f6b6c87bee.ex0', 'zeusbin_a4b50f49282e1025cddcf48d9a529e3b.ex0', 'zeusbin_3df592ae0f7115f49733244a525f4086.ex0', 'zeusbin_bc8ba3fe0b4eb6c8b365586d8addb467.ex0', 'zeusbin_2e75edd32a29ec16d11262dfbf62208e.ex0', 'zeusbin_d5dbf706348461c51dec54839d98faba.ex0', 'zeusbin_b510b94b9f53f746cf2ac63a4887610d.ex0', 'zeusbin_c2f7af739eeb3af73bf29079698cc680.ex0', 'zeusbin_0eb1a89c34a10f73dad3ce1b64df5478.ex0', 'zeusbin_3230109e119bebc2ce88398233da481a.ex0', 'zeusbin_794b5a5e2016c1dd4dab15c856acf149.ex0', 'zeusbin_9f448cdd0cdec13a5b5b0d49cffbad39.ex0', 'zeusbin_83e975d6ed89a0d4ba3705255100c29c.ex0', 'zeusbin_9cb116a3a8a818a989743ce37cb06c61.ex0', 'zeusbin_daf371079da180a6aec0bc8efb0220b3.ex0', 'zeusbin_94afbd5997b83ff971f01041612c9499.ex0', 'zeusbin_e45a28362174e677e4f1193a987d7195.ex0', 'zeusbin_9e2a7344a95c96ce32f841ad2cc6151f.ex0', 'zeusbin_ac688f5a09634855f7b7d78e9f5c1127.ex0', 'zeusbin_c5b8c10912ad3d8f827f75f97ab7f658.ex0', 'zeusbin_1810747d5861b524cb82b7ad843b4857.ex0', 'zeusbin_4b38093308f8f3e30c1f4950df6bebe7.ex0', 'zeusbin_41b7b3c2b154b19fb9310623cf0cea82.ex0', 'zeusbin_596c71354fbb521f1248b7d03860fc60.ex0', 'zeusbin_f9cfa96e45235f629fadac4dd9f0a694.ex0', 'zeusbin_68dbb46c3cc7a6ef510141d0c877ee9a.ex0', 'zeusbin_f418179746055f7e3f2403275fac1124.ex0', 'zeusbin_f767bf3602c50834aa20d159fa9c157f.ex0', 'zeusbin_947b6973b004e308f11ab6b43a548d91.ex0', 'zeusbin_bfca2500da1d489d82a3429dac82ceb7.ex0', 'zeusbin_b9a2d3dea532c67e04ece627b060d7e4.ex0', 'zeusbin_d131caa8469351546093108493ef772d.ex0', 'zeusbin_aab9ad233f2fade74bfb697ec789fe06.ex0', 'zeusbin_9542e87a3d0ca17aae615687afdd7df4.ex0', 'zeusbin_f53952356bdb605b468be1aff0b14657.ex0', 'zeusbin_a30aa6318a9e32d092d534ce8cb43da8.ex0', 'zeusbin_80f16882e1710cefe5563d9333305c46.ex0', 'zeusbin_3cebaa12cc5a8ddb48df145711603e06.ex0', 'zeusbin_4d1e91fb1be8167beaf1450f97724262.ex0', 'zeusbin_c2db1d8a98a3ed94c829db62c7e1b03e.ex0', 'zeusbin_e94de39e9a23550f4a8a18e6cf77323b.ex0', 'zeusbin_d12611d7a32da067d99320ab9b7fa9f7.ex0', 'zeusbin_3c31e3c00d5d61d5fe555dcf0062faaa.ex0', 'zeusbin_acea7eb3caa13865ee594bfcb66bc1da.ex0', 'zeusbin_8dcc9b7410966bb194246bcfb95072e4.ex0', 'zeusbin_baba375a61462ee9709b97734e07e0a4.ex0', 'zeusbin_081169ab5851ede5cb28a6a31a60ebf7.ex0', 'zeusbin_a666507eb415499a752f7eca3a2ab6e2.ex0', 'zeusbin_2f75341b152a2f643d4cedb4c1735bba.ex0', 'zeusbin_c250088ac007107a32de0f43e1df96d7.ex0', 'zeusbin_ebcc87f90d32f74274e2e0c3dd80af3e.ex0', 'zeusbin_8445f7c9e2a6334a75565df19edd2547.ex0', 'zeusbin_8f6bc3e648e519ed3de2352f133c12ae.ex0', 'zeusbin_76ee8e715d6379b69d44f8b2252978c8.ex0', 'zeusbin_65e8c7eccd61f045f532fd1b650e59e8.ex0', 'zeusbin_a4d9c971cf1f1bb84e0bd6f5926cba17.ex0', 'zeusbin_0a64f9e0e3751f1c931149c54a19c707.ex0', 'zeusbin_9d72467e076bd55fc80190888271bb6e.ex0', 'zeusbin_0ea8e12d4577a1130929e3f5bf9516c2.ex0', 'zeusbin_23602f9738396bde85702708abdb680f.ex0', 'zeusbin_ccd521a92b6fc588680f66d17adfb098.ex0', 'zeusbin_91c58770a756a250b755402fc76196d9.ex0', 'zeusbin_1ec4e0824cd4db663431077e5a75a0b2.ex0', 'zeusbin_79086e86b7cc79d747c5ce509f272770.ex0', 'zeusbin_89ef278fa84fa096a8346ed44d1c95eb.ex0', 'zeusbin_511a14bfac54e045f77c217dca831e5d.ex0', 'zeusbin_aff6441dc71c129c94f4bf587cbfcb4c.ex0', 'zeusbin_13970218b70dbdcc66b540ea22850e6d.ex0', 'zeusbin_1c03054c2dba6e6c2afdb7127c3fc6ce.ex0', 'zeusbin_67f6b656d4be8e0411a1faf1f83ea768.ex0', 'zeusbin_3f04aac5af6c98b86374d0ef3448aa4e.ex0', 'zeusbin_fc5c9e3e23c60046feb12c4826d5c156.ex0', 'zeusbin_763b9bdba331d3acd485795c2abe60c1.ex0', 'zeusbin_6f60a655091072c6b3a2e6cb1e42fa60.ex0', 'zeusbin_1403bbbd1fdd46615da65eadc087b4ae.ex0', 'zeusbin_8e77206bd7f9ebebcc1f4838c7daee80.ex0', 'zeusbin_4a1e22a60fd5503716a22aae81a87959.ex0', 'zeusbin_39dad48dfb8825e2b4ee418325c7d34a.ex0', 'zeusbin_cfb2198e961bf2266bce2f7ec2fea413.ex0', 'zeusbin_d3a3d497b4137ece54806fae6f35773c.ex0', 'zeusbin_f0e68b81cef4b212106782fe3bf030c8.ex0', 'zeusbin_a92f523c7fa6d0f127d6b955e1634e16.ex0', 'zeusbin_bac1fb5bfd89a2f325e1cb9f7bb6052e.ex0', 'zeusbin_d89e609a1b1137706fee406c163dc4e3.ex0', 'zeusbin_70074dc839aebb9cf30afb394686a531.ex0', 'zeusbin_b57f414c90dcf460810fd416df9ed05c.ex0', 'zeusbin_f021b9d4d5a94bf872718543b7c220d0.ex0', 'zeusbin_93da2f658b51c993bf2c531f338f8486.ex0', 'zeusbin_1713ec16071184b4bee3e5b2a369e8ef.ex0', 'zeusbin_935b7ed1ecfd63afe8b67a0fee429ac6.ex0', 'zeusbin_29374e035394f0d92e84c56d1585c638.ex0', 'zeusbin_971a6c8098046f0032caf836167419f3.ex0', 'zeusbin_f3ec8fffe8517fa007c0e8231cd703be.ex0', 'zeusbin_24e903fc99804f45e909be5065d4d073.ex0', 'zeusbin_a6cad59bbeaeda5dfbd71dc363b48263.ex0', 'zeusbin_25aa82ea68e6d9cd6afe9d449b6ac6bc.ex0', 'zeusbin_2bebd8fcdda41632da7a75893bbab647.ex0', 'zeusbin_122ce7df059d582dbb0324cb15abeb57.ex0', 'zeusbin_8382c82a20a26be6331f3648a94caf4d.ex0', 'zeusbin_a3da8ee3fd39ac044f806bfcc775afed.ex0', 'zeusbin_f4131be5d90d7d8ba3c1381a60f73d93.ex0', 'zeusbin_68c7476d5b08a102859a21a4b7923db5.ex0', 'zeusbin_b491fe7b916bf7036d0d044411649e49.ex0', 'zeusbin_4de74498b8c78a917af0969d801ae92c.ex0', 'zeusbin_d98f808bff65271397a46e52b8620570.ex0', 'zeusbin_d5199359c53678314f474d82ff6f107e.ex0', 'zeusbin_7c771352c8bb53ef8e8d41c3573a9575.ex0', 'zeusbin_5432faabb1a3ca6172e1419d41a9abd9.ex0', 'zeusbin_1dcd59b45b3d97a2e243d7966b67c288.ex0', 'zeusbin_98cf4569154285cf536dc9b5292a1cc5.ex0', 'zeusbin_7f6ccc203a6242dc1018040aacbbdb7e.ex0', 'zeusbin_c0dc9cf472e1b10fe2bb31b0048a5bce.ex0', 'zeusbin_eedbf70991823fc1de1afbc9b580d4e5.ex0', 'zeusbin_d1227dfa7b66efa8987373b3964c5c9f.ex0', 'zeusbin_7fdd7f89a473ca15e48d453955f9c60b.ex0', 'zeusbin_8055e6e6466020d719f1c6d611df38b5.ex0', 'zeusbin_39034f600a1efd84038a63ec90c16630.ex0', 'zeusbin_2a8ef18219ac2a853461a7a88e460bd1.ex0', 'zeusbin_c8cf97e67981c8df8b1d1b6fb993ddb5.ex0', 'zeusbin_d9219548ac5a921fb08da421ac413f65.ex0', 'zeusbin_d4291afdb00326100cca41119e71edd2.ex0', 'zeusbin_38cee6a2904b1f521039b47693825cca.ex0', 'zeusbin_ee7ef9e27a1c47bd2cc4a64888d10150.ex0', 'zeusbin_990e50357f07ad1d0ed612eb00d46723.ex0', 'zeusbin_a4d9e44de6171819e3fc8f5ce5f497df.ex0', 'zeusbin_aec8e08a1bd185b95f955c218ad4d31a.ex0', 'zeusbin_72505cb57f3aa49d2ddccb825a690969.ex0', 'zeusbin_72cafcd80a9baaca1dacd6fa66bd0260.ex0', 'zeusbin_0bdea24bead430c14df16cd6d8d3432b.ex0', 'zeusbin_db2fa262d7759b984c49509b8958a419.ex0', 'zeusbin_e646b3aa3b3f6544ae2a17f8e7274cbc.ex0', 'zeusbin_45b87b40ea0fa331177659966e9cf6c3.ex0', 'zeusbin_f34c96c3911f54f9bcb626267b91d562.ex0', 'zeusbin_2580e6e7e4f9ae510b428c32d9308495.ex0', 'zeusbin_bd323f8565e03439a61aad6debfed3e4.ex0', 'zeusbin_1493ef39903f6b5ad9790f5b396df1f3.ex0', 'zeusbin_9a2e420883d9335f2a42bcabff8ddaa2.ex0', 'zeusbin_44782f8ab7fbc857f58bd8ba1ddc2bb2.ex0', 'zeusbin_ad0c4d764e6b98ad4e21fc1e67bc8492.ex0', 'zeusbin_31469895e330575bdab654e2d965407c.ex0', 'zeusbin_5c3e8cc842dd13c2478166ef1eb3478b.ex0', 'zeusbin_91756c2a0f3208d88aff8f5a3bd0beb1.ex0', 'zeusbin_2d44b6bb04ed33ca97c9752e61a94ba8.ex0', 'zeusbin_e123e0a30976489e21fa0665a6c947ca.ex0', 'zeusbin_1ea24d61d809cc43ee34dba4dd611f04.ex0', 'zeusbin_77967ae40bfc7667dd1afc9703b85ec8.ex0', 'zeusbin_a3a648c3344c9c554a004750d5781736.ex0', 'zeusbin_dfe30d17d27b170c9b4fcb6ef58f8ce9.ex0', 'zeusbin_603b27dded1969ec50e6eed73d20f7e5.ex0', 'zeusbin_f3766a1d9cae987262a7c80c0839e7bb.ex0', 'zeusbin_4f8de33caeb7e365503ca4b4854dec1a.ex0', 'zeusbin_7f90a90bd11bad7e9bb9e7b4cf6ef55b.ex0', 'zeusbin_1864fcb6b9bbb9a2bf4238ef5de88748.ex0', 'zeusbin_b63b43567db117660875093533aef34d.ex0', 'zeusbin_2471b9f54c33088b5273d97ac3b51569.ex0', 'zeusbin_5298f1fd6b300223f6bcdbc1fa89c2c0.ex0', 'zeusbin_17e167a93858ac67451f5a57441acf5a.ex0', 'zeusbin_d4ccf9dd3b26d1506df7eabf145ef779.ex0', 'zeusbin_d33d9c3c4a2cef8895a5d28b52615df2.ex0', 'zeusbin_dc553651897e5646212e4d9e1779b60b.ex0', 'zeusbin_45d06e8d111ef3b6177ed266c87f4c77.ex0', 'zeusbin_206b7c11655f0b9091acb91f1e99098b.ex0', 'zeusbin_4fc7a47fa138fb0b2e752f706ce19b13.ex0', 'zeusbin_781c588beaef532aeb6f9b6657116d41.ex0', 'zeusbin_bdfaf34914f8c63b3b0b964b5580e1ac.ex0', 'zeusbin_9435d754a040a8e7497929037b8df104.ex0', 'zeusbin_849de8cdc9a93095ae76074dd847d1a3.ex0', 'zeusbin_a0feaa368eeadcd8b0cc99b97011e488.ex0', 'zeusbin_efd9b894f5c44b8de8edb4a4096483c2.ex0', 'zeusbin_1a2756859919c1fc115ec847419040a0.ex0', 'zeusbin_10b70e30b032a9702c41ba6a162f1c28.ex0', 'zeusbin_c68f73e43a3a4783ad820388368bd2bd.ex0', 'zeusbin_403082ccbaa5449b3a95f70c473f95ec.ex0', 'zeusbin_dca799224a76a848eb825fb9bcd8245a.ex0', 'zeusbin_10b6fa36c5d9376ff60fbfa29c4763f2.ex0', 'zeusbin_e04a202c593ef8bcc84343160eec03e6.ex0', 'zeusbin_70f58736b654ec144a0a313935dd76b6.ex0', 'zeusbin_77e988603763066a7fa8c82d3e14a957.ex0', 'zeusbin_a53114d82049953043100cbbc000ce27.ex0', 'zeusbin_33cb742c77ac12ce46731a490b3cf3d1.ex0', 'zeusbin_80e128c3e00ccabe02544a9c80fa0870.ex0', 'zeusbin_92f73630afc3d1592d81958ceab7fd9b.ex0', 'zeusbin_d0691292846c04217448e3e173856ddb.ex0', 'zeusbin_77b65c03926005a50428c5f44bbb6980.ex0', 'zeusbin_3e451b1251f7b664b6819fc0fb5ac842.ex0', 'zeusbin_3f35ba6822a88def4de560fdc8c6ee18.ex0', 'zeusbin_85ab0c4663fdb92462a568ec71843c58.ex0', 'zeusbin_98ba48b77c5c8fdbbd758f362d67abcf.ex0', 'zeusbin_81051bcc2cf1bedf378224b0a93e2877.ex0', 'zeusbin_14a18b30c40f5a4fafe08e0c21cc5844.ex0', 'zeusbin_f65a157487abd9b89ecf9f3e9ca780a6.ex0', 'zeusbin_967ac9d799c5271ee242a86ac5b1b6a4.ex0', 'zeusbin_93bc735a36cce2decc55c2119815763d.ex0', 'zeusbin_9116aab0cde5ddf4ad0ec9d4b08fcdf3.ex0', 'zeusbin_1a2cf41df6e72d9f81a503973a86ae1e.ex0', 'zeusbin_05c0902326ddafe6b489fced673b666b.ex0', 'zeusbin_89664e7b868dd503817ec94767af22e1.ex0', 'zeusbin_2a65efb38d698446605aa0f82fee2402.ex0', 'zeusbin_a790138cf667eb4e7a22b80cfdb23606.ex0', 'zeusbin_938494d130d63306aa1d0b3be7857f19.ex0', 'zeusbin_733b2176633b4ecacb38b54a9787fa90.ex0', 'zeusbin_acf87ef9761204661d48cb9266162769.ex0', 'zeusbin_ca518f8447f04f0992713065f3fc3445.ex0', 'zeusbin_f30225355a7a52101b4ac7b1f29c63d6.ex0', 'zeusbin_df26462de8cba06de75aa03b89bd9af3.ex0'}

In [5]:
unpackedFiles = {'zeusbin_cf585f2785990937d3d0b1750216df11.ex0', 'zeusbin_44782f8ab7fbc857f58bd8ba1ddc2bb2.ex0', 'zeusbin_ebc76f2f21d462941f33d1962769fc9c.ex0', 'zeusbin_2d8a1f78f8a6e9f11042322ed64143af.ex0', 'zeusbin_2bebd8fcdda41632da7a75893bbab647.ex0', 'zeusbin_c5b8c10912ad3d8f827f75f97ab7f658.ex0', 'zeusbin_4b11ffde267d271b264da95b3ca89a7e.ex0', 'zeusbin_6d008d3d7c6925bec035d4dad8d47b66.ex0', 'zeusbin_e7df872dbc5381f3596ea5470b4fdf35.ex0', 'zeusbin_d9dbb28c6af64869b79359e020642efe.ex0', 'zeusbin_d996109a926ef4092b37288dd030d2e3.ex0', 'zeusbin_7c771352c8bb53ef8e8d41c3573a9575.ex0', 'RtlUpd64.exe', 'zeusbin_31469895e330575bdab654e2d965407c.ex0', 'zeusbin_2e75edd32a29ec16d11262dfbf62208e.ex0', 'WmiApSrv.exe', 'zeusbin_8ea3c9fd270c1dd58805b444a599b069.ex0', 'zeusbin_12c1d32952f88b64990babf52936d023.ex0', 'SynLenovoGestureMgr.exe', 'WindowPicker.exe', 'PnPUnattend.exe', 'zeusbin_3a02cb0a4bd6936206590da902db5ffb.ex0', 'zeusbin_8ef7ab8b343a4cacd4027a4b990c623d.ex0', 'zeusbin_0b748a4f8636a80ea12015507b6ae1bc.ex0', 'zeusbin_13970218b70dbdcc66b540ea22850e6d.ex0', 'RtkAudioService64.exe', 'zeusbin_d63d8f3339feb3e3077223ce0029f74d.ex0', 'zeusbin_04a52158af1ce913ed5c7e174f406049.ex0', 'zeusbin_04aa0d2c110d62d24cb92622755cfacd.ex0', 'zeusbin_e4287db3afa84062d5c804af7108c652.ex0', 'TSWbPrxy.exe', 'wow64_microsoft-windows-setupapi_31bf3856ad364e35_10.0.15063.0_none_505ddd3c336d55b8_wowreg32.exe_94fc2d06', 'zeusbin_4e1aa174471703ff67f7344ad744cffb.ex0', 'zeusbin_f525a5decafcdf99ae44cd76fe9158b6.ex0', 'zeusbin_0ea8e12d4577a1130929e3f5bf9516c2.ex0', 'zeusbin_14a18b30c40f5a4fafe08e0c21cc5844.ex0', 'zeusbin_dca799224a76a848eb825fb9bcd8245a.ex0', 'print.exe', 'scrcons.exe', 'zeusbin_3075c65a964d58c2c17596c5af6713bf.ex0', 'wininst-6.0.exe', 'zeusbin_0ed58ea5eb0d4e63ff06ca0ee07dea23.ex0', 'zeusbin_77be5a8ea34c47eb803459d91a3730fc.ex0', 'tzsync.exe', 'plasrv.exe', 'zeusbin_2a8ef18219ac2a853461a7a88e460bd1.ex0', 'zeusbin_14d294fbfef36c063b96fcfb0d849d46.ex0', 'zeusbin_715a907da639533bbb27ecbced1b03fb.ex0', 'zeusbin_d5cede2be86794a4618b50e74821c01d.ex0', 'zeusbin_2f3b9a2525a91f3c06a54332b0074e21.ex0', 'perfhost.exe', 'zeusbin_6ed709ceac4f15ea0b0542d3aedcf3cb.ex0', 'zeusbin_92f73630afc3d1592d81958ceab7fd9b.ex0', 'zeusbin_a790138cf667eb4e7a22b80cfdb23606.ex0', 'zeusbin_677a4256e0a7e45ab337d5dc9fb89c2a.ex0', 'zeusbin_8a92fa240436cfcca099d1957e3f6e6a.ex0', 'zeusbin_8730b28e8cc20091509b617d856c94f9.ex0', 'zeusbin_8619f6afa4f4e3ac9c712fbde92eefcb.ex0', 'zeusbin_924830d720076f9366bcd6a89a392845.ex0', 'zeusbin_05c0902326ddafe6b489fced673b666b.ex0', 'zeusbin_a486edd5d966fd167f9d8fa94087913e.ex0', 'zeusbin_967ac9d799c5271ee242a86ac5b1b6a4.ex0', 'zeusbin_a976ac3bb8db42288ba101bc53e8ead6.ex0', 'zeusbin_d89e609a1b1137706fee406c163dc4e3.ex0', 'zeusbin_08d01c593a3f1c35a7595fb2294a1b8c.ex0', 'RmClient.exe', 'zeusbin_3e451b1251f7b664b6819fc0fb5ac842.ex0', 'zeusbin_dd836822645f8f2a70f6fe7b54235187.ex0', 'zeusbin_e164aabe89fcd8d92040a15f76c995c9.ex0', 'wiaacmgr.exe', 'zeusbin_fce31ac6f04df886b5c06f3c50ad7729.ex0', 'sihost.exe', 'zeusbin_8d096ee74a1df2862c89a5845673ed58.ex0', 'SnippingTool.exe', 'zeusbin_ee7ef9e27a1c47bd2cc4a64888d10150.ex0', 'zeusbin_d9d8d6a282e30fdad5ef55266c0d4111.ex0', 'zeusbin_6b187ac48c93d272bd192a4b8e416905.ex0', 'zeusbin_87bb648f5c59fc75a0b55d4e1a72a262.ex0', 'zeusbin_2580e6e7e4f9ae510b428c32d9308495.ex0', 'UI0Detect.exe', 'zeusbin_83560514a6a3183d836fabf4c69b0732.ex0', 'zeusbin_eedbf70991823fc1de1afbc9b580d4e5.ex0', 'Windows.Media.BackgroundPlayback.exe', 'zeusbin_dc5eb9a0b0adc1afbc0d10a4a6d0c567.ex0', 'zeusbin_1486e2c21a58f44ff14c9cef8da386fb.ex0', 'zeusbin_5c8875eba48586d8f1a3b3a6e86de426.ex0', 'zeusbin_6680786cb8fe6eec79232ef69c876b6e.ex0', 'zeusbin_1665a8fc9ed420a9870588348492cad4.ex0', 'zeusbin_251c4c1501ea7d073c03cfb9077ee290.ex0', 'setspn.exe', 'zeusbin_2812ac76b416428c0ed7adf7490d0133.ex0', 'zeusbin_39dad48dfb8825e2b4ee418325c7d34a.ex0', 'zeusbin_17e167a93858ac67451f5a57441acf5a.ex0', 'zeusbin_a599ce1912510f021bf490f6b6c87bee.ex0', 'PATHPING.EXE', 'zeusbin_cdd911de2b50e36566b483f3b3c1e25c.ex0', 'zeusbin_1480cca97da7ae5c2b63d176d2442ceb.ex0', 'zeusbin_61030ca7196ae04b50de66015f6f7c38.ex0', 'zeusbin_403f661916fb74386582cdb36ebd9320.ex0', 'zeusbin_849de8cdc9a93095ae76074dd847d1a3.ex0', 'zeusbin_5a06a69cb78215c99f469f8977f15d82.ex0', 'zeusbin_942ad1e140ea248918b9d7151dd0fdbd.ex0', 'zeusbin_5fc108a6f2b8424e8e310b88c886bfe9.ex0', 'zeusbin_68676213eaf9c93cc99564c7d6951cd2.ex0', 'zeusbin_990e50357f07ad1d0ed612eb00d46723.ex0', 'zeusbin_5002d30daaebde5748abf1681cd9694b.ex0', 'zeusbin_9e0f3c1c29a60a21d85b881ab9858dfe.ex0', 'zeusbin_eef6d9b841edcb0672660b53a49a4316.ex0', 'zeusbin_71f6590bcaf16d9203e9428e545bc5cb.ex0', 'zeusbin_5c3e8cc842dd13c2478166ef1eb3478b.ex0', 'sdiagnhost.exe', 'zeusbin_70e7e6413fa1957cb9dc85e8910eed65.ex0', 'zeusbin_16097db59de7255053b0ee5ff7a7b763.ex0', 'zeusbin_267eb35ffc49d8721d0c29e8a5542c8f.ex0', 'zeusbin_39034f600a1efd84038a63ec90c16630.ex0', 'zeusbin_89ef278fa84fa096a8346ed44d1c95eb.ex0', 'zeusbin_12b9f8583ec478d7bf7c4c3b10839b43.ex0', 'proxy-lookup.exe', 'zeusbin_b91fbf7db78c52aefc6506a425755440.ex0', 'zeusbin_77bf953b707cd00f5cb5130c6b308e59.ex0', 'zeusbin_da6fe0ff31cdfd4a6e861bfb7c0c5c91.ex0', 'ucsvc.exe', 'zeusbin_9a2e420883d9335f2a42bcabff8ddaa2.ex0', 'tzutil.exe', 'zeusbin_80f16882e1710cefe5563d9333305c46.ex0', 'zeusbin_4fb0dfc902c9bbea68d06c1d42866b53.ex0', 'pcalua.exe', 'SecHealthUI.exe', 'SMSvcHost.exe', 'zeusbin_aec8e08a1bd185b95f955c218ad4d31a.ex0', 'zeusbin_c4b8c4cbbd8c7215b688058cecea8a2e.ex0', 'zeusbin_68dbb46c3cc7a6ef510141d0c877ee9a.ex0', 'tcmsetup.exe', 'zeusbin_3d132253420ef5efd2f3bc99462bfa2e.ex0', 'zeusbin_a3ffae6ec54c2d923c3901cb3b90269d.ex0', 'zeusbin_1a2cf41df6e72d9f81a503973a86ae1e.ex0', 'pcwrun.exe', 'zeusbin_9f448cdd0cdec13a5b5b0d49cffbad39.ex0', 'zeusbin_cacc70f6b95519e2d9a1c76dce3f7ca3.ex0', 'zeusbin_603b27dded1969ec50e6eed73d20f7e5.ex0', 'zeusbin_8f16094f40acc7425f5f036cf97d05d8.ex0', 'zeusbin_de95c43229e5b0ba4a284683ccce2d78.ex0', 'zeusbin_91756c2a0f3208d88aff8f5a3bd0beb1.ex0', 'zeusbin_e172d09613dc8e32b1448ae800a10f86.ex0', 'zeusbin_ea041dae5e21d491f01d7113674de6df.ex0', 'zeusbin_4de9efe2683504d5b5ee297372526151.ex0', 'zeusbin_8e065372f9a85d580742e296a0110114.ex0', 'zeusbin_cd43fa8c663e6f0049c35c23a1d06a6e.ex0', 'zeusbin_9e2a7344a95c96ce32f841ad2cc6151f.ex0', 'zeusbin_72505cb57f3aa49d2ddccb825a690969.ex0', 'WFS.exe', 'zeusbin_0cfab1f28f6751dee14ff014bdbc3ac2.ex0', 'zeusbin_5ce5b500d22146ccaf1976bb2a52e6da.ex0', 'zeusbin_c1d417f2143b222de395b813eb951bb6.ex0', 'zeusbin_f3ec8fffe8517fa007c0e8231cd703be.ex0', 'XboxIdp.exe', 'ResetEngine.exe', 'zeusbin_09b22d4e11817e2652b205686c57841c.ex0', 'SynTPEnh.exe', 'zeusbin_d2e40f6239c358d083d147dd1d9196dc.ex0', 'zeusbin_a4ae956fb4e42c5f2eb3029cf38476f7.ex0', 'RpcPing.exe', 'zeusbin_a460c6f9ae5540fca87033a6de47cda5.ex0', 'zeusbin_d0691292846c04217448e3e173856ddb.ex0', 'zeusbin_3cebaa12cc5a8ddb48df145711603e06.ex0', 'zeusbin_b24a4a367060d83cea54f1a26b7f9532.ex0', 'PresentationFontCache.exe', 'setupugc.exe', 'zeusbin_403082ccbaa5449b3a95f70c473f95ec.ex0', 'unlodctr.exe', 'zeusbin_03fb56bf933d7f0a81d37a599517602a.ex0', 'zeusbin_74de8cebc0b66c7516e06a35398fe0e0.ex0', 'zeusbin_dfe30d17d27b170c9b4fcb6ef58f8ce9.ex0', 'unregmp2.exe', 'zeusbin_20f60acdf4b4bf46e1351b0530991dcd.ex0', 'ServiceWorkerHost.exe', 'zeusbin_f3766a1d9cae987262a7c80c0839e7bb.ex0', 'zeusbin_9d3f74140b5f7e957b2b9c6ec4a0cba4.ex0', 'zeusbin_a92f523c7fa6d0f127d6b955e1634e16.ex0', 'zeusbin_d3b35fa888ed3bd1e5783c1af652bb66.ex0', 'zeusbin_4fc7a47fa138fb0b2e752f706ce19b13.ex0', 'regedit.exe', 'zeusbin_d1227dfa7b66efa8987373b3964c5c9f.ex0', 'zeusbin_c6ae73fb4cd26ef9473b0bc7fca681a9.ex0', 'remsh.exe', 'zeusbin_3302ed4cef0e8af6e7dae40b011ea772.ex0', 'zeusbin_c7e70eb5f665eb5a32101deed7d14e4b.ex0', 'zeusbin_1da73f1c22739a13311f6e9807b3341b.ex0', 'zeusbin_aff6441dc71c129c94f4bf587cbfcb4c.ex0', 'zeusbin_ec9fafb71cfec5ba384c4d342f2a76f4.ex0', 'PilotshubApp.exe', 'zeusbin_8029dc7e003e25cd7b7488ddc97fbb24.ex0', 'zeusbin_57af12ad1138fd6fc45809adca5c7748.ex0', 'zeusbin_a93dfbff5735c7afee4857c2a432a488.ex0', 'zeusbin_c174ca2d54ee1d2f2439a44ac6b19e09.ex0', 'zeusbin_c154d321487d8d9502bcaa739bba49ae.ex0', 'zeusbin_e5ca06d9055a3d5f2af769c54ec4136b.ex0', 'zeusbin_dc553651897e5646212e4d9e1779b60b.ex0', 'zeusbin_ccd521a92b6fc588680f66d17adfb098.ex0', 'zeusbin_d9219548ac5a921fb08da421ac413f65.ex0', 'SpeechToTextOverlay64-Retail.exe', 'zeusbin_51bb2b2a8a4315793e5c8075946ee30b.ex0', 'PaintStudio.View.exe', 'zeusbin_1377860c072645b87de4c2b95fda5a31.ex0', 'zeusbin_d835bc682e092f4c968b9512f99081b5.ex0', 'zeusbin_c1c2ac433b106b57080789ce4bcb89b9.ex0', 'TokenBrokerCookies.exe', 'zeusbin_04c235c1dd6a91f86d72b785e85d2692.ex0', 'zeusbin_cdd1c50b94586ca8aa26591b5831c4b2.ex0', 'zeusbin_72cafcd80a9baaca1dacd6fa66bd0260.ex0', 'zeusbin_318e5df01d5ebe86798731f1c33a6578.ex0', 'zeusbin_9a3d52668c155c36b059c92054c789d5.ex0', 'rmttpmvscmgrsvr.exe', 'schtasks.exe', 'zeusbin_362895a0d94394f9b98e0d61235cdb6b.ex0', 'SystemSettings.exe', 'zeusbin_149a14112e95576ebc24ffb8e28261df.ex0', 'ThumbnailExtractionHost.exe', 'ssh-agent.exe', 'zeusbin_9490cf8184989d89502c9c7624572f75.ex0', 'zeusbin_da27b8578c4e8ada1cfa17fb25fa161f.ex0', 'zeusbin_763b9bdba331d3acd485795c2abe60c1.ex0', 'zeusbin_bed20d7bf94d4a837e675557bf6bbd3e.ex0', 'zeusbin_114446798966209506ed21c9ff926d72.ex0', 'zeusbin_e45a28362174e677e4f1193a987d7195.ex0', 'zeusbin_9b749134695a6e28e29cf96fca17f11b.ex0', 'zeusbin_baba375a61462ee9709b97734e07e0a4.ex0', 'zeusbin_1dcd59b45b3d97a2e243d7966b67c288.ex0', 'zeusbin_70cedaff5a8ea3a95c73a3986e0e5c87.ex0', 'proquota.exe', 'zeusbin_78efdd02b00ad78fac2e14f0014cc4bc.ex0', 'zeusbin_8ef75794ed565e2512fd0a54df04e990.ex0', 'zeusbin_aa9a41cd6e8d8e2a59ad6b06666fd2d0.ex0', 'zeusbin_935b7ed1ecfd63afe8b67a0fee429ac6.ex0', 'zeusbin_d57672d26036ebd1586a05734d5be6bc.ex0', 'zeusbin_98ba48b77c5c8fdbbd758f362d67abcf.ex0', 'zeusbin_53f2772061376525ce614fdad8c72111.ex0', 'zeusbin_487b86fe3cd9cd0e48da7134c0857567.ex0', 'zeusbin_89664e7b868dd503817ec94767af22e1.ex0', 'zeusbin_a6cad59bbeaeda5dfbd71dc363b48263.ex0', 'sapisvr.exe', 'TRACERT.EXE', 'wmlaunch.exe', 'zeusbin_f4131be5d90d7d8ba3c1381a60f73d93.ex0', 'zeusbin_aff3977dbd5a3a8b075d61c99c10c076.ex0', 'zeusbin_2420fb71d32b61b886177b20b4159a7f.ex0', 'zeusbin_10b6fa36c5d9376ff60fbfa29c4763f2.ex0', 'zeusbin_2a65efb38d698446605aa0f82fee2402.ex0', 'zeusbin_91add03073c644bfd35732fc43810189.ex0', 'zeusbin_2279de832112c4aec0972cf688c64216.ex0', 'zeusbin_90b612502670ee8a453ba190ec6e75a5.ex0', 'zeusbin_77e988603763066a7fa8c82d3e14a957.ex0', 'zeusbin_74da39ea8ba4fd490bf2f666a3a3ab05.ex0', 'zeusbin_33cb742c77ac12ce46731a490b3cf3d1.ex0', 'zeusbin_74f2c955e576fa1856702f5a5a70dffe.ex0', 'zeusbin_57e06fdb4bb8db21c2e6b8a0017a8378.ex0', 'zeusbin_4a1e22a60fd5503716a22aae81a87959.ex0', 'zeusbin_77967ae40bfc7667dd1afc9703b85ec8.ex0', 'zeusbin_68b329da9893e34099c7d8ad5cb9c940.ex0', 'zeusbin_e123e0a30976489e21fa0665a6c947ca.ex0', 'zeusbin_d95f29db5d8933fd8894dc13b8bed726.ex0', 'Time.exe', 'zeusbin_df7818ef37eafcbf7f95e3f262f921b7.ex0', 'zeusbin_38cee6a2904b1f521039b47693825cca.ex0', 'zeusbin_ce0dc1ab104d69ef12337cb60e100501.ex0', 'wordpad.exe', 'zeusbin_f8292ab24afb9afb1d2c5f2a37dbc0ae.ex0', 'zeusbin_72461bfee0808356fe9f0e8573af7650.ex0', 'zeusbin_24e903fc99804f45e909be5065d4d073.ex0', 'zeusbin_d5dbf706348461c51dec54839d98faba.ex0', 'zeusbin_fd813555c5b26fc719e782420828fd67.ex0', 'zeusbin_3e65032595fae332524a94fc0aa52edb.ex0', 'zeusbin_cb69538063aab7be10edeb4a122fd97a.ex0', 'zeusbin_0bdea24bead430c14df16cd6d8d3432b.ex0', 'zeusbin_79e704e49bcbc5d9f30fba9fdaf3e4b8.ex0', 'zeusbin_b1c06fc49699bb178ec078add0d336cc.ex0', 'smss.exe', 'zeusbin_bb62c9fabe4ac9bca4d851d1261c279c.ex0', 'zeusbin_e4c3ab8bc9a382e281cba79961a4d6d5.ex0', 'zeusbin_f34c96c3911f54f9bcb626267b91d562.ex0', 'zeusbin_8055e6e6466020d719f1c6d611df38b5.ex0', 'zeusbin_1c5ac7502a3bb3e74459fc50af0e5221.ex0', 'zeusbin_1fbfc4e242da54b1b605a22df9ea36fd.ex0', 'zeusbin_23602f9738396bde85702708abdb680f.ex0', 'zeusbin_d992a9460b72067dbccb75a1cce02639.ex0', 'zeusbin_68c7476d5b08a102859a21a4b7923db5.ex0', 'zeusbin_c89343eb3555eb425cd8d1c6c837889e.ex0', 'zeusbin_9e1b39a820e86f426475bb6651293164.ex0', 'ReAgentc.exe', 'provlaunch.exe', 'zeusbin_77b65c03926005a50428c5f44bbb6980.ex0', 'SecureAssessmentBrowser.exe', 'zeusbin_c265cbc422defd00cb2c0e5671cb407c.ex0', 'zeusbin_c835af3a25b44f4f6243f9dfaa330844.ex0', 'UevAgentPolicyGenerator.exe', 'zeusbin_ebcc87f90d32f74274e2e0c3dd80af3e.ex0', 'TsWpfWrp.exe', 'zeusbin_c8624da6815dc4451d0b717bd9e6e34a.ex0', 'zeusbin_193d7be9ff2a18b10dcd75072334a54f.ex0', 'zeusbin_5a8108521f05eeea5f4b0e3043edf782.ex0', 'PurchaseApp.exe', 'zeusbin_b1abbf464d7c4a4ac15acd234ee5484f.ex0', 'TapiUnattend.exe', 'zeusbin_b0fc8ef8bee2c79eb37f6880b06381d5.ex0', 'zeusbin_a3a648c3344c9c554a004750d5781736.ex0', 'zeusbin_695b42f564eec7c3527e322da77e2947.ex0', 'zeusbin_5b4f372d1a638d780ab160448eecf65d.ex0', 'zeusbin_cbd463bf72564da7f7631f133b7d7a16.ex0', 'SkypeHost.exe', 'zeusbin_8c1c88dda615ec741ae8880acc5e3934.ex0', 'zeusbin_d98f808bff65271397a46e52b8620570.ex0', 'zeusbin_1febf89c92b7b4ddd697d4c22ae0d16a.ex0', 'zeusbin_0eb1a89c34a10f73dad3ce1b64df5478.ex0', 'zeusbin_c9e0fb04d933a98c6f1ca6a4cd52a0f8.ex0', 'zeusbin_91c58770a756a250b755402fc76196d9.ex0', 'wpnpinst.exe', 'zeusbin_acea7eb3caa13865ee594bfcb66bc1da.ex0', 'zeusbin_34a40ad0c257045d11ddde855b45fa66.ex0', 'zeusbin_885ebb6a97edc31d95ff361a68a7b96e.ex0', 'zeusbin_8f331ea71f67a4a4e3e8e1463be0c92d.ex0', 'wininst-7.1.exe', 'WebCache.exe', 'zeusbin_3d76812520c74833d3a16c2a4849160e.ex0', 'zeusbin_70074dc839aebb9cf30afb394686a531.ex0', 'zeusbin_e0b8a8a12af18f15e69cac5edc631b50.ex0', 'zeusbin_25aa82ea68e6d9cd6afe9d449b6ac6bc.ex0', 'zeusbin_c617fe4b0c9d11af6cb8e1bcaa61cd54.ex0', 'zeusbin_f67d176eaa0d2df796fd3daa3e6ccabc.ex0', 'verclsid.exe', 'zeusbin_6172e6278c2f0a012a6aca5a5069b221.ex0', 'zeusbin_20dda594b415f1c1b3b1215de5d6db05.ex0', 'zeusbin_12707d8923ab9d86079d89480de97c3f.ex0', 'zeusbin_b63b43567db117660875093533aef34d.ex0', 'zeusbin_4c48875c464b2b5944d7959bc8a5cb0d.ex0', 'zeusbin_df26462de8cba06de75aa03b89bd9af3.ex0', 'shutdown.exe', 'zeusbin_83e975d6ed89a0d4ba3705255100c29c.ex0', 'UevTemplateConfigItemGenerator.exe', 'zeusbin_1810747d5861b524cb82b7ad843b4857.ex0', 'UsoClient.exe', 'zeusbin_174f31e04800c1384a1a6c2f88c42eee.ex0', 'zeusbin_38029aaa03ca6f20bb0d67f789aa4923.ex0', 'wsqmcons.exe', 'zeusbin_e808a684802c67aa513a8e1a4056643a.ex0', 'zeusbin_68f7f2dd30dc480579557780333a8bf3.ex0', 'zeusbin_2e17b7b37870469d65b938ceb76d457c.ex0', 'zeusbin_081169ab5851ede5cb28a6a31a60ebf7.ex0', 'zeusbin_ca518f8447f04f0992713065f3fc3445.ex0', 'zeusbin_cfb2198e961bf2266bce2f7ec2fea413.ex0', 'zeusbin_bf7041da35376da532ac8be3279aeaf5.ex0', 'zeusbin_7f6ccc203a6242dc1018040aacbbdb7e.ex0', 'zeusbin_8dcc9b7410966bb194246bcfb95072e4.ex0', 'zeusbin_b57f414c90dcf460810fd416df9ed05c.ex0', 'tabcal.exe', 'zeusbin_bebe31d157e5fa35a9a2be3e28852f5c.ex0', 'zeusbin_64f5a6c9c725cd7b1986164bd3afe0d1.ex0', 'zeusbin_ad3d733620917bf92e121db01a529fcf.ex0', 'pwlauncher.exe', 'zeusbin_10c6568ab0e5b7bf4e4aa591dae8bfdb.ex0', 'zeusbin_33ea76477b0e08215f03c6e383bf8359.ex0', 'SecEdit.exe', 'zeusbin_904553e47f7ab4f6e1bfa7502327303e.ex0', 'Squirrel.exe', 'Update.exe', 'timeout.exe', 'zeusbin_1a2756859919c1fc115ec847419040a0.ex0', 'SkypeApp.exe', 'zeusbin_a8a3e535cc3b54363b7d5764a71302b6.ex0', 'zeusbin_dd84d7393e8c2f7d140eaf07e518fff2.ex0', 'upnpcont.exe', 'WindowsActionDialog.exe', 'zeusbin_94d85dd9e0f5f16ca285552afe30a174.ex0', 'subst.exe', 'zeusbin_85ab0c4663fdb92462a568ec71843c58.ex0', 'zeusbin_68b80a17f61a19570fbbb587e715303c.ex0', 'zeusbin_3f35ba6822a88def4de560fdc8c6ee18.ex0', 'zeusbin_d490a173c8ba38d0390126d02ce773c8.ex0', 'zeusbin_a4d9c971cf1f1bb84e0bd6f5926cba17.ex0', 'zeusbin_d15305d7a4e34e02489c74a5ef542f36.ex0', 'zeusbin_408f3ceb086db1f75b758c9a8800e848.ex0', 'WinBioDataModelOOBE.exe', 'zeusbin_4521bef0e4020f8c79253ee97af97cd7.ex0', 'zeusbin_a4b50f49282e1025cddcf48d9a529e3b.ex0', 'zeusbin_1403bbbd1fdd46615da65eadc087b4ae.ex0', 'WhatsNew.Store.exe', 'zeusbin_6a855c130d08a08295246d73a3d79af9.ex0', 'zeusbin_c69999d0c0bc520e366f551727d5937d.ex0', 'zeusbin_148f868f3718ac83a9b77bb53ecf7cbe.ex0', 'WSReset.exe', 'zeusbin_1b4c833e1a18d0c5240d6694cbd5f25f.ex0', 'zeusbin_8e77206bd7f9ebebcc1f4838c7daee80.ex0', 'zeusbin_1b00452dd589cf71591a5f9943c0ab93.ex0', 'zeusbin_f5389a8ee6d0860d9eaf8ddcb044be73.ex0', 'WSManHTTPConfig.exe', 'zeusbin_a30aa6318a9e32d092d534ce8cb43da8.ex0', 'zeusbin_f986515cd2d1a0d2611ec84b3f4006fd.ex0', 'pythonw.exe', 'zeusbin_8c9952a9adb62b36381d29aa51901eb9.ex0', 'RdpSaUacHelper.exe', 'write.exe', 'regedt32.exe', 'zeusbin_a830e6782611e8e29b468918d83c54c3.ex0', 'zeusbin_62d609084169dbb95f4ca333435e05d0.ex0', 'zeusbin_83cfcd0e8bd653c8aaf2bdfcd936edb1.ex0', 'zeusbin_c8cf97e67981c8df8b1d1b6fb993ddb5.ex0', 'wiawow64.exe', 'ServiceModelReg.exe', 'zeusbin_69ef65e895d8768d145fa20ce37c3701.ex0', 'zeusbin_8ddef4e68e8495c0dc77a5b55655dbdd.ex0', 'zeusbin_b2b1325cbfcc52be62e38fb99713b1f3.ex0', 'zeusbin_81051bcc2cf1bedf378224b0a93e2877.ex0', 'zeusbin_4ecf15b28c9a99860f8129cec979a4a5.ex0', 'zeusbin_e265b2c11f928c5465cffa5188a8c3ca.ex0', 'zeusbin_9cb116a3a8a818a989743ce37cb06c61.ex0', 'zeusbin_59fc670b36704fdcf6bfec41f21b7853.ex0', 'zeusbin_16ab81b2c997bff0154a9b7f92f6503d.ex0', 'zeusbin_67afbc78df726b6ba198f856843f84a9.ex0', 'zeusbin_83eefb4ae7dfde2b265549e2c60ce4fa.ex0', 'zeusbin_a182b445ff83d46d593b39f99a791a9a.ex0', 'zeusbin_6395582467cce9bef97fc9a93bff739e.ex0', 'TP.exe', 'zeusbin_1493ef39903f6b5ad9790f5b396df1f3.ex0', 'PrintDialog.exe', 'zeusbin_fc921d17d55525e066a3ed9d9d664f3f.ex0', 'zeusbin_c68f73e43a3a4783ad820388368bd2bd.ex0', 'wowreg32.exe', 'zeusbin_0b02e6141fdeb045a5475fc423d2dafe.ex0', 'sc.exe', 'zeusbin_98cf4569154285cf536dc9b5292a1cc5.ex0', 'zeusbin_129e271c36cbd083c1251aa08f48dc25.ex0', 'secinit.exe', 'Solitaire.exe', 'zeusbin_a2c0171a4739242b695976e73376c0da.ex0', 'zeusbin_3df592ae0f7115f49733244a525f4086.ex0', 'zeusbin_3230109e119bebc2ce88398233da481a.ex0', 'zeusbin_5298f1fd6b300223f6bcdbc1fa89c2c0.ex0', 'zeusbin_0504941110dd0449ce2de9e829b6316e.ex0', 'VaultCmd.exe', 'zeusbin_b2193bcd49f155e5aca54a0fdb36d75b.ex0', 'zeusbin_d4ccf9dd3b26d1506df7eabf145ef779.ex0', 'zeusbin_a25263c96b548b76031d96b43fe46b08.ex0', 'zeusbin_909e644c2e21b73b476e9ae888111584.ex0', 'zeusbin_079019625e44d3802d94f83cc73b83f6.ex0', 'zeusbin_b220a7bc97e21d1a13a08ead5cf90116.ex0', 'zeusbin_94c7358e46c9ebc7c90af9e7dc3dd8ed.ex0', 'RemindersShareTargetApp.exe', 'zeusbin_db2fa262d7759b984c49509b8958a419.ex0', 'zeusbin_2b8e48c443c15c33986b754b76bdc8fb.ex0', 'zeusbin_f0ca232e22a5b045741417df32176892.ex0', 'zeusbin_8e316d144823751eb0af131845684a62.ex0', 'zeusbin_7adb5762997379b2c007f7e93839dfe3.ex0', 'zeusbin_5432faabb1a3ca6172e1419d41a9abd9.ex0', 'Register-CimProvider.exe', 'shrpubw.exe', 'zeusbin_d5199359c53678314f474d82ff6f107e.ex0', 'TpmInit.exe', 'zeusbin_bac1fb5bfd89a2f325e1cb9f7bb6052e.ex0', 'zeusbin_8e04cfba1315c3e122a4eeaa8489f4a0.ex0', 'zeusbin_af2c35c0ca5ee1097f3a782b1bb1cfa4.ex0', 'zeusbin_d4291afdb00326100cca41119e71edd2.ex0', 'PeopleApp.exe', 'zeusbin_89f79e297e439304cc26519eab772bb3.ex0', 'zeusbin_bc8ba3fe0b4eb6c8b365586d8addb467.ex0', 'zeusbin_eea8c0976aa9d464d0bd0112247acf96.ex0', 'zeusbin_f9cfa96e45235f629fadac4dd9f0a694.ex0', 'zeusbin_f6016a7e2df1774fcd35459302b9f060.ex0', 'zeusbin_33098ef529c83d81e9b82ab7a0a03252.ex0', 'UevTemplateBaselineGenerator.exe', 'zeusbin_0097e7ebc48af545d0758c9f2fb6882e.ex0', 'zeusbin_a0faffef6ffdb13cfe95408e9ca8a0ea.ex0', 'zeusbin_8fab18d966599fc977cb75f1216ad327.ex0', 'RpgMakerSaveEdit.exe', 'userinit.exe', 'zeusbin_3ef0eea1cc53703cabd283fcc4b0e57a.ex0', 'zeusbin_2de8e8f9761dfb0b01b871155de46bbb.ex0', 'zeusbin_6333c5c2c7091c5c26aca372b041755d.ex0', 'zeusbin_a36ac29bdce56a9822fb94c76f5d6303.ex0', 'zeusbin_bb37ff859cce2d7c5faa6b6b1301835b.ex0', 'zeusbin_ee5d6f9e4625be005dab2b599c3ba968.ex0', 'zeusbin_97675eb3f268048604dc5155511a2a4d.ex0', 'zeusbin_e834f5a83db82b75205eb8617aff2db9.ex0', 'zeusbin_49790b8b6c07e330aa343555cbba048a.ex0', 'zeusbin_c9f958989c1f1db0979d386795effa0d.ex0', 'zeusbin_bfd636829520bc53852f7bcaf0530ff5.ex0', 'WindowsUpdateElevatedInstaller.exe', 'zeusbin_94afbd5997b83ff971f01041612c9499.ex0', 'WpcTok.exe', 'zeusbin_45b87b40ea0fa331177659966e9cf6c3.ex0', 'ShellExperienceHost.exe', 'zeusbin_f3ea5c3e8991ebe87bd2549d64e09765.ex0', 'zeusbin_9750ab9b939fada0691388a13e55c156.ex0', 'zeusbin_a3da8ee3fd39ac044f806bfcc775afed.ex0', 'XboxApp.exe', 'zeusbin_c250088ac007107a32de0f43e1df96d7.ex0', 'zeusbin_a92d6e402e300976c4bd9c7c37742e21.ex0', 'zeusbin_781c588beaef532aeb6f9b6657116d41.ex0', 'zeusbin_101eac98f8226e3e3ee8901be06eb6d9.ex0', 'zeusbin_bf27ac5c832d929e94066a1320ba3e95.ex0', 'zeusbin_ce0e4be45919462647e19f526a26b6e9.ex0', 'zeusbin_565d51366441e800b424bb3927f99f97.ex0', 'zeusbin_68bed8dbfefd986c2c8b66d9eb290181.ex0', 'WFServicesReg.exe', 'XamlViewer_v0300.exe', 'wscadminui.exe', 'zeusbin_f021b9d4d5a94bf872718543b7c220d0.ex0', 'WPDShextAutoplay.exe', 'zeusbin_cf628a9f53c82036fe644a39de7292ee.ex0', 'zeusbin_e04a202c593ef8bcc84343160eec03e6.ex0', 'zeusbin_3a5864ecb258afb33ecc964a093b0336.ex0', 'zeusbin_f7f7e615ee8c7c3aff2cb69e8e4b1bee.ex0', 'zeusbin_7f90a90bd11bad7e9bb9e7b4cf6ef55b.ex0', 'zeusbin_93bc735a36cce2decc55c2119815763d.ex0', 'zeusbin_8e472d33b1616a3e9a036d99e2177b08.ex0', 'zeusbin_794b5a5e2016c1dd4dab15c856acf149.ex0', 'snmptrap.exe', 'zeusbin_9c8c705051f0bf9d6ba9be1745090ef6.ex0', 'zeusbin_76ee8e715d6379b69d44f8b2252978c8.ex0', 'zeusbin_27d2ed6678e1a6f736e1c9a1dd76e0ba.ex0', 'zeusbin_647a2c43e508323da8f3a013e0de1c7a.ex0', 'zeusbin_0ffa350c0bc96ef1368b3d2279f3378f.ex0', 'zeusbin_9116aab0cde5ddf4ad0ec9d4b08fcdf3.ex0', 'WindowsCamera.exe', 'windeploy.exe', 'zeusbin_2471b9f54c33088b5273d97ac3b51569.ex0', 'zeusbin_2d44b6bb04ed33ca97c9752e61a94ba8.ex0', 'zeusbin_15495577934562a08d8ab9a11be3ba62.ex0', 'zeusbin_5206c204a880215fcaec7104017807e1.ex0', 'zeusbin_4526c2dcf11bcc1b2db868d96b8f4463.ex0', 'zeusbin_1e233a7613b8c8ae9e077b6a217f9f62.ex0', 'zeusbin_e646b3aa3b3f6544ae2a17f8e7274cbc.ex0', 'zeusbin_803f6e85d809e601360c22cb80a3e8f2.ex0', 'zeusbin_d131caa8469351546093108493ef772d.ex0', 'zeusbin_10bb6b5fc6678e51ab9aa090e3b23595.ex0', 'zeusbin_385239507f00bbaffdc1136fd7bbe73b.ex0', 'zeusbin_e1cd3a4bf381a26ad785357ba4b099de.ex0', 'zeusbin_59c440c05b16118b4e71be067f0f0849.ex0', 'zeusbin_1c03054c2dba6e6c2afdb7127c3fc6ce.ex0', 'zeusbin_814ae4263ecb3ec7f6c70f4dcf5747e9.ex0', 'zeusbin_a46ff85867847cdbefa8c343975e71ca.ex0', 'zeusbin_49eba7529172f6dbdf8a87b712cdb645.ex0', 'zeusbin_e4f0fd1fe6a1903245a2d709f6e55bfb.ex0', 'zeusbin_086089fc98f3afbc9ff2cda8dda80942.ex0', 'zeusbin_93da2f658b51c993bf2c531f338f8486.ex0', 'zeusbin_1864fcb6b9bbb9a2bf4238ef5de88748.ex0', 'WorkFolders.exe', 'zeusbin_3f04aac5af6c98b86374d0ef3448aa4e.ex0', 'zeusbin_8382c82a20a26be6331f3648a94caf4d.ex0', 'zeusbin_e5f5bc6c6576e43b11cb0ade1823d3d3.ex0', 'ROUTE.EXE', 'verifier.exe', 'zeusbin_1ec4e0824cd4db663431077e5a75a0b2.ex0', 'zeusbin_70f58736b654ec144a0a313935dd76b6.ex0', 'zeusbin_ef4564964d6b66bd2f8db5c633475b67.ex0', 'zeusbin_2f75341b152a2f643d4cedb4c1735bba.ex0', 'PING.EXE', 'zeusbin_23004c619a695b5d78fb435dd6f2442c.ex0', 'zeusbin_fc990e6dfa5c53850c6d2f36b0c2ab48.ex0', 'wmprph.exe', 'zeusbin_dec1a97cd5d24430e2d9b2cfb8f6864d.ex0', 'zeusbin_1aeb1def75f13a5b590586606575cbd1.ex0', 'zeusbin_43cd2fd09b821afeb1a96933f9f1d6a8.ex0', 'zeusbin_b0145344ea4ad267efbc653b2992974c.ex0', 'SpeechModelDownload.exe', 'zeusbin_6709f99c91e762884d3e45c8c0d0fa98.ex0', 'unsecapp.exe', 'zeusbin_369e4e031925b1e341482e630be93731.ex0', 'zeusbin_79189537e13971cabc64a47a98454909.ex0', 'zeusbin_f767bf3602c50834aa20d159fa9c157f.ex0', 'zeusbin_660181d71cf48cf4fc6ba435a4ec0dd7.ex0', 'zeusbin_4e6d7f70f9e23a774910e6649c127078.ex0', 'zeusbin_0a64f9e0e3751f1c931149c54a19c707.ex0', 'PrintDialogHost3D.exe', 'zeusbin_e4c0248128fe8a362cf06c680c05365d.ex0', 'zeusbin_0f50312e20f6b5f2ff1ec5bbdfca4ee3.ex0', 'zeusbin_981bc042cd2ba4f34fc0de3d927a4a73.ex0', 'zeusbin_a4d9e44de6171819e3fc8f5ce5f497df.ex0', 'zeusbin_bdfaf34914f8c63b3b0b964b5580e1ac.ex0', 'zeusbin_67f6b656d4be8e0411a1faf1f83ea768.ex0', 'zeusbin_a03f9e8a3e8067139ad6e0170a573915.ex0', 'ssh-add.exe', 'zeusbin_e94de39e9a23550f4a8a18e6cf77323b.ex0', 'zeusbin_c8eb57fad3a9663f72fc3a6038ff0e7f.ex0', 'zeusbin_9c8e0593ba342152a39c1a6d38740218.ex0', 'zeusbin_e46be0d5e5baa48ade7378a27fc3041f.ex0', 'wmpnscfg.exe', 'UevAppMonitor.exe', 'provtool.exe', 'powershell.exe', 'zeusbin_ad0c4d764e6b98ad4e21fc1e67bc8492.ex0', 'zeusbin_d270cb1b95d1fee59e32ae19121f82d6.ex0', 'zeusbin_756447e177fc3cc39912797b7ecb2f92.ex0', 'winrshost.exe', 'WSCollect.exe', 'zeusbin_2a662f283c6d9fceb1c473aaf9175bd0.ex0', 'zeusbin_bfca2500da1d489d82a3429dac82ceb7.ex0', 'pipanel.exe', 'RemotePosWorker.exe', 'SystemSettingsBroker.exe', 'zeusbin_971a6c8098046f0032caf836167419f3.ex0', 'zeusbin_9542e87a3d0ca17aae615687afdd7df4.ex0', 'UpgradeResultsUI.exe', 'zeusbin_acf87ef9761204661d48cb9266162769.ex0', 'zeusbin_252d59c765189e7a5ad1309f49724188.ex0', 'zeusbin_b67dc1cbcb23ccef1a0328f152bf53a5.ex0', 'zeusbin_93ac7005ad34a78f4f081b8a72885e54.ex0', 'zeusbin_b375c578a7cc01de1f8fe103ef74a0e1.ex0', 'zeusbin_c0dc9cf472e1b10fe2bb31b0048a5bce.ex0', 'zeusbin_c9cc24c3633661cd1739513f0cc0aa84.ex0', 'zeusbin_29374e035394f0d92e84c56d1585c638.ex0', 'zeusbin_6f60a655091072c6b3a2e6cb1e42fa60.ex0', 'zeusbin_440af60cdcd6addc29ba04616715738b.ex0', 'RDSPnf.exe', 'zeusbin_b863338d9d2b603faa4469f3b1ff5e6a.ex0', 'zeusbin_0ee321b3e4a47dba44f7bb14fb6b9efe.ex0', 'zeusbin_a0feaa368eeadcd8b0cc99b97011e488.ex0', 'rasautou.exe', 'zeusbin_506ae5000c07ca11d643da15cb8173ba.ex0', 'zeusbin_c1669f678a5942d314611e79e7deeade.ex0', 'WpcUapApp.exe', 'zeusbin_b9a2d3dea532c67e04ece627b060d7e4.ex0', 'vdsldr.exe', 'sdbinst.exe', 'runas.exe', 'zeusbin_daf371079da180a6aec0bc8efb0220b3.ex0', 'zeusbin_6dd76848f68b9acd2d805163c85f8d5f.ex0', 'zeusbin_28e7837eaa3d39e6f76ee16136d02aa2.ex0', 'zeusbin_7fdd7f89a473ca15e48d453955f9c60b.ex0', 'zeusbin_41b7b3c2b154b19fb9310623cf0cea82.ex0', 'zeusbin_4d1e91fb1be8167beaf1450f97724262.ex0', 'zeusbin_10b70e30b032a9702c41ba6a162f1c28.ex0', 'zeusbin_4b38093308f8f3e30c1f4950df6bebe7.ex0', 'vssadmin.exe', 'SynReflash.exe', 'RtsCM64.exe', 'zeusbin_511a14bfac54e045f77c217dca831e5d.ex0', 'zeusbin_79086e86b7cc79d747c5ce509f272770.ex0', 'zeusbin_f418179746055f7e3f2403275fac1124.ex0', 'srdelayed.exe', 'zeusbin_65e8c7eccd61f045f532fd1b650e59e8.ex0', 'zeusbin_8f6bc3e648e519ed3de2352f133c12ae.ex0', 'SrTasks.exe', 'replace.exe', 'wlrmdr.exe', 'zeusbin_ab6588df7bf80307986cabc764080478.ex0', 'zeusbin_b491fe7b916bf7036d0d044411649e49.ex0', 'zeusbin_80e128c3e00ccabe02544a9c80fa0870.ex0', 'zeusbin_5f6a44d8559707942f29f9fc0c7efcdc.ex0', 'zeusbin_b2f144fc2891acad31d92593e564ae0b.ex0', 'zeusbin_f90e52d0e6aa39ad91c94ffef5502975.ex0', 'zeusbin_122e910f20c37f121060d24fcb16d783.ex0', 'zeusbin_3caaab49b856d8f02b7959e08eb33086.ex0', 'zeusbin_1db382dae4f2b3a84117245695697f97.ex0', 'zeusbin_62169455fd9b0908eb51191e38217c4c.ex0', 'tpmvscmgrsvr.exe', 'rasdial.exe', 'zeusbin_c2db1d8a98a3ed94c829db62c7e1b03e.ex0', 'zeusbin_887cd71f5c491f50d8b317e4b6f42c1a.ex0', 'zeusbin_10531148760a98a5441ce93f4432561c.ex0', 'zeusbin_729b9e4d7de555236f66455bf1186833.ex0', 'zeusbin_697fce300c9878d15e4306093b75be3d.ex0', 'zeusbin_4d7f45666f2af51474beefc0838b351f.ex0', 'zeusbin_947b6973b004e308f11ab6b43a548d91.ex0', 'zeusbin_3028e0638275ba8979f551812e0793c6.ex0', 'zeusbin_938494d130d63306aa1d0b3be7857f19.ex0', 'zeusbin_fc5c9e3e23c60046feb12c4826d5c156.ex0', 'RAVBg64.exe', 'zeusbin_517cbe2453b6d2e4cb0418eaffa94a53.ex0', 'zeusbin_f30225355a7a52101b4ac7b1f29c63d6.ex0', 'zeusbin_a53114d82049953043100cbbc000ce27.ex0', 'setup_wm.exe', 'zeusbin_912e4c3a98b7b77fa8cadb8ed77b9846.ex0', 'zeusbin_c2f7af739eeb3af73bf29079698cc680.ex0', 'WallpaperHost.exe', 'recover.exe', 'ssh.exe', 'zeusbin_9657febc71ccada0e9cfae5871f589f3.ex0', 'zeusbin_b510b94b9f53f746cf2ac63a4887610d.ex0', 'zeusbin_733b2176633b4ecacb38b54a9787fa90.ex0', 'zeusbin_f65a157487abd9b89ecf9f3e9ca780a6.ex0', 'zeusbin_596c71354fbb521f1248b7d03860fc60.ex0', 'zeusbin_ac1924b156a0a7c213eb174a9c2249e7.ex0', 'WinStore.App.exe', 'TCPSVCS.EXE', 'zeusbin_a7a466e0aac0b7b79f0c15f77dec4df0.ex0', 'SystemResetPlatform.exe', 'python.exe', 'zeusbin_144b49b2b46195e9864aa9f317199bfb.ex0', 'UserAccountControlSettings.exe', 'zeusbin_fa519bfd49ba82acd616cb12a82f998c.ex0', 'zeusbin_9435d754a040a8e7497929037b8df104.ex0', 'zeusbin_2c9067ac187db40dbf1948f50db1f185.ex0', 'WsatConfig.exe', 'zeusbin_8f6917df893caa43cd84b9c42df3f8b4.ex0', 'zeusbin_750e5d1723422782e1d41e31f2287965.ex0', 'zeusbin_520f2694b5b775af0da1ce71bc4fb39b.ex0', 'zeusbin_ac688f5a09634855f7b7d78e9f5c1127.ex0', 'zeusbin_9098c057091100cfb3d80ee99a7add4b.ex0', 'zeusbin_d333bc2fbb7c425f673f00ed928973a0.ex0', 'zeusbin_a666507eb415499a752f7eca3a2ab6e2.ex0', 'zeusbin_d33d9c3c4a2cef8895a5d28b52615df2.ex0', 'zeusbin_1e1450ea831c148d34c8d3d75bc2eaf0.ex0', 'zeusbin_206b7c11655f0b9091acb91f1e99098b.ex0', 'zeusbin_f0e68b81cef4b212106782fe3bf030c8.ex0', 'zeusbin_3c31e3c00d5d61d5fe555dcf0062faaa.ex0', 'zeusbin_a204bb60062b4570214d3565634fa257.ex0', 'zeusbin_d12611d7a32da067d99320ab9b7fa9f7.ex0', 'zeusbin_f53952356bdb605b468be1aff0b14657.ex0', 'UserOOBEBroker.exe', 'wow64_microsoft-windows-appidcore_31bf3856ad364e35_10.0.15063.0_none_4a395d1c23946704_appidtel.exe_b664fbc5', 'wow64_microsoft-windows-rasautodial_31bf3856ad364e35_10.0.15063.0_none_2b428241d2829ac6_rasautou.exe_477abe34', 'zeusbin_122ce7df059d582dbb0324cb15abeb57.ex0', 'systray.exe', 'winhlp32.exe', 'zeusbin_c22cb47aa0ec07a506c8e54478e4d83e.ex0', 'XblGameSaveTask.exe', 'zeusbin_c6ca82f4af53c4ba08cfeea57f27fc16.ex0', 'RdpSaProxy.exe', 'zeusbin_bd323f8565e03439a61aad6debfed3e4.ex0', 'zeusbin_1dfe9cb8538a519092870e2c7bace705.ex0', 'VSSVC.exe', 'zeusbin_ad3cfca22c1bc9610651ec2bc93a4e69.ex0', 'spaceman.exe', 'zeusbin_51dd42665519e92bd4fda4790afda309.ex0', 'zeusbin_61b05811d680bc6490ca47bd2c90823a.ex0', 'zeusbin_12b46389be1acce5142619192e3d33da.ex0', 'zeusbin_bec7595c4b5377921474293bb6075311.ex0', 'stordiag.exe', 'regsvr32.exe', 'zeusbin_aab9ad233f2fade74bfb697ec789fe06.ex0', 'zeusbin_4de74498b8c78a917af0969d801ae92c.ex0', 'zeusbin_1713ec16071184b4bee3e5b2a369e8ef.ex0', 'zeusbin_4f8de33caeb7e365503ca4b4854dec1a.ex0', 'zeusbin_2ffac0ebcbbe9f32df017d7f099020f8.ex0', 'zeusbin_62316b2f638dd4c43b9da30935ba5261.ex0', 'zeusbin_efd9b894f5c44b8de8edb4a4096483c2.ex0', 'zeusbin_d978af223bf2e440261db4d5de52ddea.ex0', 'zeusbin_9d72467e076bd55fc80190888271bb6e.ex0', 'zeusbin_52ab4a86cf8402d4ea6e86b45360d081.ex0', 'zeusbin_05a9cd49dd0b3b6cc1a7e48464999b9e.ex0', 'zeusbin_79462f574b7b022869050089f7d3f0d2.ex0', 'zeusbin_ca21d6561951fc87cbb0c3a55c9159e6.ex0', 'SpeechRuntime.exe', 'XBox.TCUI.exe', 'wuapihost.exe', 'user.exe', 'zeusbin_8445f7c9e2a6334a75565df19edd2547.ex0', 'zeusbin_3b7b7b62445f5ce8f930cb474ecbfe37.ex0', 'zeusbin_1ea24d61d809cc43ee34dba4dd611f04.ex0', 'zeusbin_d3a3d497b4137ece54806fae6f35773c.ex0', 'zeusbin_77b2c7e0c6a0b5ce2ccf46cbdb8ec1a3.ex0', 'zeusbin_45d06e8d111ef3b6177ed266c87f4c77.ex0', 'zeusbin_b9d5e13b7aee8645c73b63c1d73e7550.ex0', 'syskey.exe'}

In [6]:
len(unpackedFiles)
numberOfFiles = len(text_files)

In [7]:
len(unpackedFiles)

753

In [8]:
dataDir = "/Users/Phaniteja/desktop/RA/latestTestFiles/"
text_files = []
for key in unpackedFiles:
    text_files.append(key)
print(text_files)

['zeusbin_5432faabb1a3ca6172e1419d41a9abd9.ex0', 'zeusbin_942ad1e140ea248918b9d7151dd0fdbd.ex0', 'zeusbin_bfca2500da1d489d82a3429dac82ceb7.ex0', 'subst.exe', 'proquota.exe', 'spaceman.exe', 'zeusbin_df7818ef37eafcbf7f95e3f262f921b7.ex0', 'zeusbin_1486e2c21a58f44ff14c9cef8da386fb.ex0', 'zeusbin_29374e035394f0d92e84c56d1585c638.ex0', 'zeusbin_2420fb71d32b61b886177b20b4159a7f.ex0', 'zeusbin_e164aabe89fcd8d92040a15f76c995c9.ex0', 'zeusbin_d4ccf9dd3b26d1506df7eabf145ef779.ex0', 'setupugc.exe', 'zeusbin_d9dbb28c6af64869b79359e020642efe.ex0', 'zeusbin_bc8ba3fe0b4eb6c8b365586d8addb467.ex0', 'zeusbin_3c31e3c00d5d61d5fe555dcf0062faaa.ex0', 'zeusbin_1713ec16071184b4bee3e5b2a369e8ef.ex0', 'zeusbin_68b80a17f61a19570fbbb587e715303c.ex0', 'zeusbin_a4d9c971cf1f1bb84e0bd6f5926cba17.ex0', 'zeusbin_91c58770a756a250b755402fc76196d9.ex0', 'zeusbin_a53114d82049953043100cbbc000ce27.ex0', 'zeusbin_8fab18d966599fc977cb75f1216ad327.ex0', 'tcmsetup.exe', 'zeusbin_4ecf15b28c9a99860f8129cec979a4a5.ex0', 'regsvr32.

In [9]:
malwareCount=[]
benignCount= []
for i in range(0,len(text_files)):
    if text_files[i][0:4]=="zeus":
        malwareCount.append(text_files[i])
    else:
        benignCount.append(text_files[i])

In [10]:
text_files = malwareCount[0:182]+benignCount

In [11]:
len(text_files)

364

In [12]:
#All the initilizations in the project 
unpacked_files = []
count = 0
mostFrequentNgrams = []
length_Ngram_Dictionaries = []
global_Dictionary_ngrams = {}
file_Dictionary_ngram = []

In [13]:
#definition for the accuracy of the given dataset
def findAccuracies(groundTruth,predictedLabel):
    real_Label = []
    for i in range(0,len(groundTruth)):
        if groundTruth[i][:4]=="zeus":
            real_Label.append(1)
        else:
            real_Label.append(0)
    print((real_Label),(predictedLabel))
    return v_measure_score(ground_Truth,list(Predicted_Label))*100

In [14]:
#Defining entropy
def entropy(s):
    p, lns = Counter(s), float(len(s))
    return -sum( count/lns * math.log(count/lns, 2) 
                for count in p.values())

In [15]:
#creates the ngram dictionary and returns it
def createTensor(file,ngram):
    #create the ngrams first and then add them in the dictionary 
    nGram_dictionary = {}
    for i in range(0,len(file),ngram):
        ngram_word=file[i:i+ngram]
        if ngram_word not in global_Dictionary_ngrams:
            global_Dictionary_ngrams[ngram_word]=1
        if ngram_word in nGram_dictionary:
            nGram_dictionary[ngram_word][0]+=1
            nGram_dictionary[ngram_word][1].append(location_Calculator(i,len(file)))
        else:
            nGram_dictionary[ngram_word]=[]
            nGram_dictionary[ngram_word].append(1)
            loc = location_Calculator(i,len(file))
            nGram_dictionary[ngram_word].append([loc])  
    return nGram_dictionary 

In [16]:
#given the file length and its ngram it returns the location in file
def location_Calculator(i,length):
    location_word = i/length*100
    #print(location_word)
    if location_word <=10:
        return 10
    elif location_word <=20:
        return 20
    elif location_word <=30:
        return 30
    elif location_word <=40:
        return 40
    elif location_word <=50:
        return 50
    elif location_word <=60:
        return 60
    elif location_word <=70:
        return 70
    elif location_word <=80:
        return 80
    elif location_word <=90:
        return 90
    return 100

In [17]:
#creates the tensor from the given dictionaries
def makeATensor(globalDict,fileDict):
    #print(globalDict,fileDict)
    tensor = np.zeros((numberOfFiles,len(globalDict),11))
    count_file = 0
    total_Ngrams = 0 
    for f in text_files:
        theFile = open(dataDir+f, "rb").read()
        theFile = theFile[:1000]
        for i in range(0,len(theFile),5):
            total_Ngrams+=1
            ngram_word=theFile[i:i+5]
            index_File = count_file
            index_Ngram = globalDict[ngram_word]
            loc_Ngram = location_Calculator(i,len(theFile))
            #print(index_File,index_Ngram,int(loc_Ngram/10))
            tensor[index_File][index_Ngram][int(loc_Ngram/10)]+=1    
        count_file +=1
    file = open("tensorTesting.txt","w") 
    file.write(str(tensor))
    file.close() 
#     print(tensor[1])
#     #x = np.arange(200).reshape((4,5,10))
#     with file('test.txt', 'w') as outfile:
#         for slice_2d in tensor:
#             np.savetxt(outfile, slice_2d)
    print("Ngrams are")
    print(total_Ngrams)
    return tensor

In [18]:
#decomposition Tecnique for the given tensor
def parafac_decomposition(tensor):
        """
        Computes a parafac decomposition of the tensor.
        This will return n rank 3 factor matrices, where n represents the 
        dimensionality of the tensor.
        :return:
        """
        decompose = KruskalTensor(tensor.shape, rank=3, regularize=1e-6, 
                                  init='nvecs', X_data=tensor)
        factors = decompose.U
        with tf.Session() as sess:
            for i in range(len(factors)):
                sess.run(factors[i].initializer)
                factors[i] = factors[i].eval()
        return factors

In [19]:
#for vizualizing and its clustering we use this function 
def k_means_clustering(factor_matrix):
        clusters = 2
        svd = TruncatedSVD(n_components=2, n_iter=20, random_state=42)
        reduced = svd.fit_transform(factor_matrix)
        kmeans = KMeans(n_clusters=clusters, random_state=0).fit(factor_matrix)
        labels_predicted = kmeans.labels_
        #print(labels_predicted)
        #print(text_files)
        #lets write the function to check the accuracy here 
        #we have 4 different categories 0-zeus 0-benign 1-zeus 0-benign
        zero_Zeus = 0
        zero_Benign = 0
        one_Zeus = 0 
        one_Benign = 0 
        for i in range(0,len(text_files)):
            #print(text_files[i][len(text_files[i])-3:],labels_predicted[i])
            if text_files[i][len(text_files[i])-3:]=="exe" and labels_predicted[i]==0:
                zero_Benign+=1
            elif text_files[i][len(text_files[i])-3:]=="exe" and labels_predicted[i]==1:
                one_Benign+=1
            elif text_files[i][len(text_files[i])-3:]=="ex0" and labels_predicted[i]==0:
                zero_Zeus+=1
            elif text_files[i][len(text_files[i])-3:]=="ex0" and labels_predicted[i]==1:
                one_Zeus+=1
            else:
                continue
        print(zero_Zeus,zero_Benign,one_Zeus,one_Benign) 
        data = [plotly.graph_objs.Scatter(x=[entry[0] for entry in reduced],
                                          y=[entry[1] for entry in reduced],
                                          mode='markers',
                                          marker=dict(color=kmeans.labels_),
                                          text = text_files
                                          )
                ]
        fig = go.Figure(data=data)
        plotly.offline.plot(fig, filename='kmeans_cluster.html')
        return text_files,labels_predicted

In [20]:
#function to create the similarity matrix for the heatmap used
def generate_cosine_similarity_matrix(matrix):
    cosine_sim = []
    for entry in matrix:
        sim = []
        for other_entry in matrix:
            sim.append(spatial.distance.cosine(entry, other_entry) * -1 + 1)
        cosine_sim.append(sim)
    return cosine_sim

In [21]:
#function to create the heatmap of the following tensor decomposition
def generate_heat_map(data, axis_labels):
    """
    Generates a heat map for the current data
    Currently only meant to support using a cosine similarity matrix
    :param data:
    :param axis_labels:
    :return:
    """
    axis_labels_abbreviated = [label[:14] for label in axis_labels]
    info = [go.Heatmap(z=data,
                       x=axis_labels_abbreviated,
                       y=axis_labels_abbreviated,
                       colorscale='Hot',
                       )]

    layout = go.Layout(title='Cosine Similarity Between Documents',
                       xaxis=dict(ticks=''),
                       yaxis=dict(ticks=''),
                       plot_bgcolor='#444',
                       paper_bgcolor='#eee'
                       )
    fig = go.Figure(data=info, layout=layout)
    plotly.offline.plot(fig, filename='heatmap.html')

In [22]:
#This the clock where we run the whole program and its the driver program!
for f in text_files:
    count+=1
    if count == 250:
        print("read file ",f)
        count = 0
    theFile = open(dataDir+f, "rb").read()
    #print(len(theFile))
    #call the tensor decomposition stuff here and create the tensor here
    dictNgrams_file = createTensor(theFile[:1000],5)
    file_Dictionary_ngram.append(dictNgrams_file)
    #print(dictNgrams_file)
    length_Ngram_Dictionaries.append(len(dictNgrams_file))
    for key in dictNgrams_file:
        if dictNgrams_file[key][0]>1:
            mostFrequentNgrams.append([key,dictNgrams_file[key]])
    #print(mostFrequentNgrams)
    #print("Next Exe File")
#print(max(length_Ngram_Dictionaries))
j = 0
for key in global_Dictionary_ngrams:
    global_Dictionary_ngrams[key]=j
    j = j+1    
#call a function which takes the global dictionary, each file dictionary and creates the tensor
tensor_Ngram = makeATensor(global_Dictionary_ngrams,file_Dictionary_ngram)
#print(global_Dictionary_ngrams)   
#print(tensor_Ngram)
#print(np.amax(tensor_Ngram))    
#print(np.sum(tensor_Ngram))
#Got the tensor here as tensor_Ngram, now lets form its factor matrices 
#and then try to cluster them 
factors_Ngrams = parafac_decomposition(tensor_Ngram)
#print(factors_Ngrams[2])
#lets send the factor matrices to k means clustering to see how the files will be clustered

k_means_clustering(factors_Ngrams[0])
#here we are done with the ploting 
# let's check how correct we are goin in the current process
#Run next cell for the accuracies
cos_sim = generate_cosine_similarity_matrix(factors_Ngrams[0])
generate_heat_map(cos_sim, text_files)

# if args['heatmap']:
#         cos_sim = tdt.generate_cosine_similarity_matrix(factors[args['axis']])
#         visualize.generate_heat_map(cos_sim, tdt.corpus_names)
#     if args['kmeans']:
#         visualize.k_means_clustering(factors[args['axis']], tdt.corpus_names, 
#                                      clusters=args['components'])



read file  verifier.exe
Ngrams are
67789
77 14 105 160


/anaconda3/lib/python3.6/site-packages/scipy/spatial/distance.py:505: RuntimeWarning:

invalid value encountered in double_scalars



## All the below blocks are just for testing purpose! Feel free to edit!

In [ ]:
findAccuracies(ground_Truth,list(Predicted_Label))

In [ ]:
createTensor("helloworld!",2) #Testing purpose for the breaking of the string 

In [ ]:
x = np.zeros((4,238,10))  # Testing purpose of tensor

In [ ]:
x = "malware.exe"
x[len(x)-3:]